In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import ast

# Load our data with warning suppressed
df_clean = pd.read_csv('data/mtg_cards_clean.csv', low_memory=False)
print(f"Loaded {len(df_clean)} cards")

# Recreate keyword features function
def create_keyword_features(df):
    important_keywords = [
        'defender', 'flying', 'vigilance', 'deathtouch', 'lifelink', 'trample',
        'haste', 'first strike', 'double strike', 'hexproof', 'indestructible',
        'proliferate', 'counter', 'artifact', 'enchantment', 'token', 'draw',
        'graveyard', 'exile', 'sacrifice', 'destroy', 'search', 'toughness',
        'power', 'enters', 'whenever', 'combat damage', 'attacks', 'skulk',
        'lose', 'gain', 'life', 'dies', 'discard', 'attacking', 'triggered',
        'ability', 'additional'
    ]
    
    keyword_matrix = []
    for _, row in df.iterrows():
        text = row['combined_text'].lower()
        features = []
        for keyword in important_keywords:
            count = text.count(keyword)
            features.append(min(count, 3))
        keyword_matrix.append(features)
    
    return np.array(keyword_matrix), important_keywords

# Color identity functions
def get_color_identity(colors_string):
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

# Create the model components
keyword_matrix, keywords = create_keyword_features(df_clean)

# Save everything
model_data = {
    'df_clean': df_clean,
    'keyword_matrix': keyword_matrix,
    'keywords': keywords,
    'get_color_identity': get_color_identity,
    'is_legal_in_deck': is_legal_in_deck
}

with open('data/mtg_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("✅ Complete model with color identity saved successfully!")

Loaded 27623 cards
✅ Complete model with color identity saved successfully!


In [7]:
# Create corrected Flask app without pickled functions
flask_app_code = '''
from flask import Flask, render_template, request, jsonify
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast

app = Flask(__name__)

# Define color functions directly in this file
def get_color_identity(colors_string):
    """Convert color string to set for easier comparison"""
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    """Check if card is legal in commander's color identity"""
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

# Load our saved model (without the functions)
print("Loading ML model...")
with open('data/mtg_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

df_clean = model_data['df_clean']
keyword_matrix = model_data['keyword_matrix']
keywords = model_data['keywords']

print(f"Model loaded! {len(df_clean)} cards available")

def find_recommendations(commander_name, num_recommendations=10):
    """Main recommendation function for web app"""
    # Find commander
    card_matches = df_clean[df_clean['name'].str.contains(commander_name, case=False, na=False)]
    
    if len(card_matches) == 0:
        return {"error": f"Commander '{commander_name}' not found"}
    
    commander_row = card_matches.iloc[0]
    commander_idx = card_matches.index[0]
    commander_colors = commander_row['colors']
    commander_vector = keyword_matrix[commander_idx].reshape(1, -1)
    
    # Calculate similarities
    similarities = cosine_similarity(commander_vector, keyword_matrix).flatten()
    all_indices = similarities.argsort()[::-1]
    
    # Filter for legal cards
    results = []
    for idx in all_indices:
        if idx == commander_idx:
            continue
            
        card_row = df_clean.iloc[idx]
        if is_legal_in_deck(card_row['colors'], commander_colors):
            results.append({
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'price': card_row.get('usd', 'N/A')
            })
            
            if len(results) >= num_recommendations:
                break
    
    return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line']
        },
        "recommendations": results
    }

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    commander = data.get('commander', '')
    num_recs = data.get('num_recommendations', 10)
    
    result = find_recommendations(commander, num_recs)
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True, port=5000)
'''

# Save the corrected Flask app
with open('app.py', 'w') as f:
    f.write(flask_app_code)

print("✅ Fixed Flask app created!")
print("Now try running: python app.py")

✅ Fixed Flask app created!
Now try running: python app.py


In [4]:
import os

# Create templates directory
os.makedirs('templates', exist_ok=True)

# Create the HTML template
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        body { font-family: Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px; }
        .header { text-align: center; margin-bottom: 30px; }
        .search-box { margin-bottom: 30px; }
        .search-box input { padding: 10px; width: 300px; font-size: 16px; }
        .search-box button { padding: 10px 20px; font-size: 16px; background: #007bff; color: white; border: none; cursor: pointer; }
        .commander-info { background: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px; }
        .recommendations { display: grid; gap: 15px; }
        .card { border: 1px solid #ddd; padding: 15px; border-radius: 8px; background: white; }
        .card-name { font-weight: bold; font-size: 18px; color: #007bff; }
        .card-type { color: #666; margin: 5px 0; }
        .card-text { margin: 10px 0; line-height: 1.4; }
        .similarity { background: #28a745; color: white; padding: 3px 8px; border-radius: 4px; font-size: 12px; }
        .colors { margin: 5px 0; }
        .color { display: inline-block; width: 20px; height: 20px; border-radius: 50%; margin-right: 5px; }
        .loading { text-align: center; font-style: italic; color: #666; }
        .error { color: #dc3545; background: #f8d7da; padding: 10px; border-radius: 4px; }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 10 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-info">
                    <h2>Commander: ${data.commander.name}</h2>
                    <p><strong>Type:</strong> ${data.commander.type}</p>
                    <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                </div>
                
                <h3>🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                <div class="recommendations">
            `;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="card">
                        <div class="card-name">${index + 1}. ${card.name} 
                            <span class="similarity">Similarity: ${(card.similarity * 100).toFixed(1)}%</span>
                        </div>
                        <div class="card-type">${card.type}</div>
                        <div class="colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                        <div class="card-text">${card.text}</div>
                    </div>
                `;
            });
            
            html += '</div>';
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the HTML template
with open('templates/index.html', 'w') as f:
    f.write(html_template)

print("✅ HTML template created in 'templates/index.html'")
print("\nReady to run the web app! 🚀")

✅ HTML template created in 'templates/index.html'

Ready to run the web app! 🚀


In [1]:
# Let's create this in a Jupyter notebook cell for better formatting control
readme_content = '''# 🧙‍♂️ MTG Commander Deck Recommender

An AI-powered Magic: The Gathering Commander deck recommendation system that suggests cards with high synergy while respecting color identity rules.

## ✨ Features

- **Content-Based Filtering**: Uses machine learning to find cards with similar mechanics and text
- **Color Identity Compliance**: Only recommends cards legal in your commander's color identity  
- **Web Interface**: Clean, user-friendly web app for easy card discovery
- **Budget Conscious**: Focuses on cards under $10
- **Keyword-Based Similarity**: Matches on important MTG mechanics like proliferate, defender, artifacts, etc.

## 🚀 How It Works

1. **Data Collection**: Gathers 4,000+ Commander-legal cards under $10 from Scryfall API
2. **Feature Extraction**: Converts card text into numerical features based on MTG keywords
3. **Similarity Calculation**: Uses cosine similarity to find mechanically similar cards
4. **Color Filtering**: Ensures all recommendations are legal in the commander's color identity
5. **Web Interface**: Presents results in an intuitive web application

## 🛠️ Technology Stack

- **Python 3.11**: Core programming language
- **scikit-learn**: Machine learning algorithms
- **TensorFlow**: ML framework (with Apple Silicon optimization)
- **Flask**: Web framework
- **Pandas**: Data manipulation
- **Scryfall API**: MTG card data source

## 🏃‍♂️ Quick Start

1. **Clone the repository**
   ```bash
   git clone https://github.com/yourusername/mtg-commander-recommender.git
   cd mtg-commander-recommender

2. **Set up Python environment**
   python3.11 -m venv mtg-env
   source mtg-env/bin/activate  # On Windows: mtg-env\\Scripts\\activate
   pip install -r requirements.txt

3. **Run data collection**
   jupyter notebook
   # Run 01_data_exploration.ipynb to collect fresh data

4. **Start app**
   python app.py

5. **Run app**
   Open your browser to http://localhost:5000

## 🧠 ML Approach
This project uses content-based filtering to recommend cards by analyzing MTG-specific keywords and mechanics rather than generic text similarity.

## 🙏 Acknowledgments

Scryfall API for MTG data
The MTG community for deck building insights

Magic: The Gathering is © Wizards of the Coast
'''

In [3]:
with open('README.md', 'w') as f:
    f.write(readme_content)
print("✅ README.md created successfully!")

✅ README.md created successfully!


In [6]:
# Create enhanced HTML template with card images
enhanced_html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        body { 
            font-family: Arial, sans-serif; 
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #f5f5f5;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { margin: 0; font-size: 2.5em; }
        .header p { margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9; }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 12px 15px; 
            width: 350px; 
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 25px;
            margin-right: 10px;
        }
        
        .search-box button { 
            padding: 12px 25px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 25px;
            cursor: pointer; 
            transition: transform 0.2s;
        }
        
        .search-box button:hover { transform: translateY(-2px); }
        
        .commander-info { 
            background: white;
            padding: 20px; 
            border-radius: 10px; 
            margin-bottom: 25px; 
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            display: flex;
            gap: 20px;
            align-items: center;
        }
        
        .commander-image {
            flex-shrink: 0;
        }
        
        .commander-image img {
            width: 200px;
            height: auto;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        }
        
        .commander-details {
            flex-grow: 1;
        }
        
        .commander-details h2 {
            margin: 0 0 10px 0;
            color: #333;
            font-size: 1.8em;
        }
        
        .recommendations { 
            display: grid; 
            grid-template-columns: repeat(auto-fill, minmax(350px, 1fr));
            gap: 20px; 
        }
        
        .card { 
            border: 1px solid #ddd; 
            border-radius: 12px; 
            background: white; 
            overflow: hidden;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: transform 0.2s, box-shadow 0.2s;
        }
        
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 15px rgba(0,0,0,0.2);
        }
        
        .card-image-container {
            position: relative;
            width: 100%;
            height: 200px;
            overflow: hidden;
            background: #f0f0f0;
        }
        
        .card-image {
            width: 100%;
            height: 100%;
            object-fit: cover;
            transition: transform 0.3s;
        }
        
        .card:hover .card-image {
            transform: scale(1.05);
        }
        
        .card-content {
            padding: 15px;
        }
        
        .card-name { 
            font-weight: bold; 
            font-size: 16px; 
            color: #333;
            margin-bottom: 8px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        
        .similarity { 
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white; 
            padding: 4px 8px; 
            border-radius: 12px; 
            font-size: 11px;
            font-weight: bold;
        }
        
        .card-type { 
            color: #666; 
            margin: 5px 0; 
            font-size: 14px;
        }
        
        .card-text { 
            margin: 10px 0; 
            line-height: 1.4; 
            font-size: 13px;
            color: #555;
        }
        
        .card-meta {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-top: 10px;
            padding-top: 10px;
            border-top: 1px solid #eee;
        }
        
        .colors { 
            font-size: 12px;
            color: #666;
        }
        
        .rarity {
            font-size: 11px;
            padding: 2px 6px;
            border-radius: 8px;
            text-transform: capitalize;
        }
        
        .rarity.common { background: #f8f9fa; color: #6c757d; }
        .rarity.uncommon { background: #e3f2fd; color: #1976d2; }
        .rarity.rare { background: #fff3e0; color: #f57c00; }
        .rarity.mythic { background: #fce4ec; color: #c2185b; }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 40px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 15px; 
            border-radius: 8px; 
            margin: 20px 0;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 100%;
            height: 100%;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
        }
        
        @media (max-width: 768px) {
            .commander-info {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 150px;
            }
            
            .search-box input {
                width: 250px;
                margin-bottom: 10px;
            }
            
            .recommendations {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-info">
                    <div class="commander-image">
                        ${data.commander.image_url ? 
                            `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>` :
                            `<div class="image-placeholder"></div>`
                        }
                    </div>
                    <div class="commander-details">
                        <h2>${data.commander.name}</h2>
                        <p><strong>Type:</strong> ${data.commander.type}</p>
                        <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                        <p><strong>Oracle Text:</strong> ${data.commander.text}</p>
                        ${data.commander.scryfall_url ? `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                    </div>
                </div>
                
                <h3>🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                <div class="recommendations">
            `;
            
            data.recommendations.forEach((card, index) => {
                const imageHtml = card.image_url ? 
                    `<img src="${card.image_url}" alt="${card.name}" class="card-image" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                    `<div class="image-placeholder"></div>`;
                
                html += `
                    <div class="card">
                        <div class="card-image-container">
                            ${imageHtml}
                        </div>
                        <div class="card-content">
                            <div class="card-name">
                                ${card.name}
                                <span class="similarity">${(card.similarity * 100).toFixed(1)}%</span>
                            </div>
                            <div class="card-type">${card.type}</div>
                            <div class="card-text">${card.text}</div>
                            <div class="card-meta">
                                <div class="colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                                <div class="rarity ${card.rarity}">${card.rarity}</div>
                            </div>
                            ${card.scryfall_url ? `<p style="margin-top: 10px; font-size: 12px;"><a href="${card.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                        </div>
                    </div>
                `;
            });
            
            html += '</div>';
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the enhanced HTML template
with open('templates/index_with_images.html', 'w') as f:
    f.write(enhanced_html_template)

# Also update the main template
with open('templates/index.html', 'w') as f:
    f.write(enhanced_html_template)

print("✅ Enhanced HTML template with card images created!")

✅ Enhanced HTML template with card images created!


In [7]:
# Update the main app.py to use the image functionality
import shutil

# Copy the enhanced app to be the main app
shutil.copy('app_with_images.py', 'app.py')
print("✅ Updated main app.py with image functionality!")

print("\nNow test your enhanced web app:")
print("1. Run: python app.py")
print("2. Open: http://localhost:5000")
print("3. Search for a commander like 'Arcades' or 'Isshin'")
print("4. You should see card images in a beautiful grid layout!")

✅ Updated main app.py with image functionality!

Now test your enhanced web app:
1. Run: python app.py
2. Open: http://localhost:5000
3. Search for a commander like 'Arcades' or 'Isshin'
4. You should see card images in a beautiful grid layout!


In [8]:
# Let's debug the image data step by step
import pandas as pd
import ast

# Load data and examine image structure
df_clean = pd.read_csv('data/mtg_cards_clean.csv', low_memory=False)

print("Debugging image display issues...")

# Check if image_uris column exists
if 'image_uris' in df_clean.columns:
    print("✅ image_uris column found")
    
    # Look at some sample image data
    sample_cards = df_clean[df_clean['image_uris'].notna()].head(5)
    
    for i, (idx, card) in enumerate(sample_cards.iterrows()):
        print(f"\n--- Card {i+1}: {card['name']} ---")
        print(f"image_uris type: {type(card['image_uris'])}")
        print(f"image_uris content: {card['image_uris'][:200]}...")
        
        # Try to parse it
        try:
            if isinstance(card['image_uris'], str):
                parsed = ast.literal_eval(card['image_uris'])
                print(f"✅ Successfully parsed")
                print(f"Available sizes: {list(parsed.keys())}")
                if 'normal' in parsed:
                    print(f"Normal image URL: {parsed['normal']}")
                else:
                    print(f"First available URL: {list(parsed.values())[0]}")
            else:
                print(f"❌ Not a string: {type(card['image_uris'])}")
        except Exception as e:
            print(f"❌ Parse error: {e}")
            
else:
    print("❌ image_uris column not found")
    print(f"Available columns: {list(df_clean.columns)}")

Debugging image display issues...
✅ image_uris column found

--- Card 1: +2 Mace ---
image_uris type: <class 'str'>
image_uris content: {'small': 'https://cards.scryfall.io/small/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221', 'normal': 'https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jp...
✅ Successfully parsed
Available sizes: ['small', 'normal', 'large', 'png', 'art_crop', 'border_crop']
Normal image URL: https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221

--- Card 2: Aarakocra Sneak ---
image_uris type: <class 'str'>
image_uris content: {'small': 'https://cards.scryfall.io/small/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jpg?1722040128', 'normal': 'https://cards.scryfall.io/normal/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jp...
✅ Successfully parsed
Available sizes: ['small', 'normal', 'large', 'png', 'art_crop', 'border_crop']
Normal image URL: https://cards.scryfall.io/normal/fron

In [9]:
def get_card_image_url_fixed(row, size='normal'):
    """Fixed function to extract card image URL"""
    try:
        # Check if image_uris exists and is not null
        if 'image_uris' not in row or pd.isna(row['image_uris']):
            return None
            
        image_uris = row['image_uris']
        
        # Handle different data types
        if isinstance(image_uris, str):
            # Try to parse JSON-like string
            try:
                image_dict = ast.literal_eval(image_uris)
            except:
                # If that fails, try json.loads
                import json
                try:
                    image_dict = json.loads(image_uris)
                except:
                    print(f"Could not parse image_uris for {row.get('name', 'unknown')}")
                    return None
        elif isinstance(image_uris, dict):
            image_dict = image_uris
        else:
            return None
        
        # Get the requested size or fallback
        if size in image_dict:
            return image_dict[size]
        elif 'normal' in image_dict:
            return image_dict['normal']
        elif 'small' in image_dict:
            return image_dict['small']
        elif image_dict:
            # Return any available image
            return list(image_dict.values())[0]
        else:
            return None
            
    except Exception as e:
        print(f"Error getting image for {row.get('name', 'unknown')}: {e}")
        return None

# Test the fixed function
print("\nTesting fixed image function:")
for i in range(min(3, len(df_clean))):
    card = df_clean.iloc[i]
    image_url = get_card_image_url_fixed(card)
    print(f"{card['name']}: {image_url}")


Testing fixed image function:
+2 Mace: https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221
Aarakocra Sneak: https://cards.scryfall.io/normal/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jpg?1722040128
Aatchik, Emerald Radian: https://cards.scryfall.io/normal/front/f/b/fbdaa29b-85ff-4a06-b27e-fcdbdfd4a3fe.jpg?1738356563


In [10]:
# Let's also test if we can access a known Scryfall image URL
import requests

# Test a known Scryfall image URL format
test_url = "https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg"

try:
    response = requests.head(test_url, timeout=5)
    print(f"Test image URL status: {response.status_code}")
    if response.status_code == 200:
        print("✅ Scryfall images are accessible")
    else:
        print("❌ Scryfall image not accessible")
except Exception as e:
    print(f"❌ Error accessing test image: {e}")

# Also check what the actual image_uris look like
print("\nActual image_uris samples:")
for i in range(3):
    if i < len(df_clean):
        card = df_clean.iloc[i]
        if pd.notna(card.get('image_uris')):
            print(f"\n{card['name']}:")
            print(f"Raw: {card['image_uris']}")

Test image URL status: 200
✅ Scryfall images are accessible

Actual image_uris samples:

+2 Mace:
Raw: {'small': 'https://cards.scryfall.io/small/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221', 'normal': 'https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221', 'large': 'https://cards.scryfall.io/large/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221', 'png': 'https://cards.scryfall.io/png/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.png?1627701221', 'art_crop': 'https://cards.scryfall.io/art_crop/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221', 'border_crop': 'https://cards.scryfall.io/border_crop/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221'}

Aarakocra Sneak:
Raw: {'small': 'https://cards.scryfall.io/small/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jpg?1722040128', 'normal': 'https://cards.scryfall.io/normal/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jpg?1722040128', 'large

In [11]:
# Test if the image URLs work
import requests

test_urls = [
    "https://cards.scryfall.io/normal/front/e/8/e882c9f9-bf30-46b6-bedc-379d2c80e5cb.jpg?1627701221",
    "https://cards.scryfall.io/normal/front/2/a/2a83882c-3e03-4e85-aaac-97fa1d08a772.jpg?1722040128"
]

print("Testing image URL accessibility:")
for i, url in enumerate(test_urls):
    try:
        response = requests.head(url, timeout=10)
        print(f"URL {i+1}: Status {response.status_code} {'✅' if response.status_code == 200 else '❌'}")
    except Exception as e:
        print(f"URL {i+1}: Error - {e}")

# The URLs look good, so let's update the Flask app with the correct image function

Testing image URL accessibility:
URL 1: Status 200 ✅
URL 2: Status 200 ✅


In [12]:
# Create the corrected Flask app with proper image handling
corrected_flask_app = '''
from flask import Flask, render_template, request, jsonify
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast
import json

app = Flask(__name__)

# Color identity functions
def get_color_identity(colors_string):
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

def get_card_image_url(row, size='normal'):
    """Extract card image URL from the image_uris field"""
    try:
        # Check if image_uris exists and is not null
        if 'image_uris' not in row or pd.isna(row['image_uris']):
            return None
            
        image_uris = row['image_uris']
        
        # Parse the string representation of the dictionary
        if isinstance(image_uris, str):
            try:
                image_dict = ast.literal_eval(image_uris)
            except:
                try:
                    image_dict = json.loads(image_uris)
                except:
                    return None
        elif isinstance(image_uris, dict):
            image_dict = image_uris
        else:
            return None
        
        # Get the requested size or fallback
        size_priority = [size, 'normal', 'small', 'large']
        for sz in size_priority:
            if sz in image_dict:
                return image_dict[sz]
        
        # Return any available image if none of the preferred sizes found
        return list(image_dict.values())[0] if image_dict else None
            
    except Exception as e:
        print(f"Error getting image for {row.get('name', 'unknown')}: {e}")
        return None

# Load enhanced model
print("Loading enhanced ML model...")
try:
    with open('data/mtg_model_enhanced.pkl', 'rb') as f:
        model_data = pickle.load(f)
    
    df_clean = model_data['df_clean']
    keyword_matrix = model_data['keyword_matrix']
    keywords = model_data['keywords']
    
    print(f"Enhanced model loaded!")
    print(f"  - {len(df_clean)} cards")
    print(f"  - {len(keywords)} keywords")
    
    # Test image extraction on first few cards
    print("Testing image extraction:")
    for i in range(3):
        card = df_clean.iloc[i]
        img_url = get_card_image_url(card)
        print(f"  {card['name']}: {'✅' if img_url else '❌'} {img_url[:50] if img_url else 'No image'}...")
    
except FileNotFoundError:
    print("Enhanced model not found, using basic model...")
    with open('data/mtg_model.pkl', 'rb') as f:
        model_data = pickle.load(f)
    df_clean = model_data['df_clean']
    keyword_matrix = model_data['keyword_matrix']
    keywords = model_data['keywords']

def find_recommendations(commander_name, num_recommendations=10):
    """Enhanced recommendation function with images"""
    # Find commander
    card_matches = df_clean[df_clean['name'].str.contains(commander_name, case=False, na=False)]
    
    if len(card_matches) == 0:
        return {"error": f"Commander '{commander_name}' not found"}
    
    commander_row = card_matches.iloc[0]
    commander_idx = card_matches.index[0]
    commander_colors = commander_row['colors']
    commander_vector = keyword_matrix[commander_idx].reshape(1, -1)
    
    # Calculate similarities
    similarities = cosine_similarity(commander_vector, keyword_matrix).flatten()
    all_indices = similarities.argsort()[::-1]
    
    # Filter for legal cards with non-zero similarity
    results = []
    for idx in all_indices:
        if idx == commander_idx:
            continue
            
        card_row = df_clean.iloc[idx]
        similarity = similarities[idx]
        
        # Only include cards with some similarity and legal color identity
        if similarity > 0 and is_legal_in_deck(card_row['colors'], commander_colors):
            card_result = {
                'name': card_row['name'],
                'similarity': float(similarity),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', '')
            }
            results.append(card_result)
            
            if len(results) >= num_recommendations:
                break
    
    return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', '')
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images v1.1"
        }
    }

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    commander = data.get('commander', '')
    num_recs = data.get('num_recommendations', 10)
    
    result = find_recommendations(commander, num_recs)
    return jsonify(result)

@app.route('/test')
def test():
    """Test endpoint to check image URLs"""
    test_card = df_clean.iloc[0]
    return jsonify({
        'name': test_card['name'],
        'image_url': get_card_image_url(test_card),
        'raw_image_uris': test_card.get('image_uris', 'Not found')
    })

if __name__ == '__main__':
    app.run(debug=True, port=5000)
'''

# Save the corrected Flask app
with open('app.py', 'w') as f:
    f.write(corrected_flask_app)

print("✅ Corrected Flask app saved!")
print("\nNow test it:")
print("1. Run: python app.py")
print("2. Check if images work by going to: http://localhost:5000/test")
print("3. Then try searching for a commander!")

✅ Corrected Flask app saved!

Now test it:
1. Run: python app.py
2. Check if images work by going to: http://localhost:5000/test
3. Then try searching for a commander!


In [13]:
# Create updated HTML template with proper image sizing
fixed_html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        body { 
            font-family: Arial, sans-serif; 
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #f5f5f5;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { margin: 0; font-size: 2.5em; }
        .header p { margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9; }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 12px 15px; 
            width: 350px; 
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 25px;
            margin-right: 10px;
        }
        
        .search-box button { 
            padding: 12px 25px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 25px;
            cursor: pointer; 
            transition: transform 0.2s;
        }
        
        .search-box button:hover { transform: translateY(-2px); }
        
        .commander-info { 
            background: white;
            padding: 20px; 
            border-radius: 10px; 
            margin-bottom: 25px; 
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            display: flex;
            gap: 20px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
        }
        
        .commander-image img {
            width: 180px;
            height: 251px;  /* MTG card aspect ratio: 180 * 1.4 = 251 */
            object-fit: cover;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        }
        
        .commander-details {
            flex-grow: 1;
        }
        
        .commander-details h2 {
            margin: 0 0 10px 0;
            color: #333;
            font-size: 1.8em;
        }
        
        .recommendations { 
            display: grid; 
            grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
            gap: 20px; 
        }
        
        .card { 
            border: 1px solid #ddd; 
            border-radius: 12px; 
            background: white; 
            overflow: hidden;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: transform 0.2s, box-shadow 0.2s;
        }
        
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 15px rgba(0,0,0,0.2);
        }
        
        .card-image-container {
            position: relative;
            width: 100%;
            height: 180px;  /* Fixed height for consistency */
            overflow: hidden;
            background: #f0f0f0;
        }
        
        .card-image {
            width: 100%;
            height: 100%;
            object-fit: contain;  /* Changed from cover to contain to show full card */
            object-position: center;
            transition: transform 0.3s;
            background: #f8f8f8;
        }
        
        .card:hover .card-image {
            transform: scale(1.05);
        }
        
        .card-content {
            padding: 15px;
        }
        
        .card-name { 
            font-weight: bold; 
            font-size: 14px; 
            color: #333;
            margin-bottom: 8px;
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            line-height: 1.2;
        }
        
        .card-name-text {
            flex-grow: 1;
            margin-right: 8px;
        }
        
        .similarity { 
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white; 
            padding: 4px 8px; 
            border-radius: 12px; 
            font-size: 11px;
            font-weight: bold;
            white-space: nowrap;
            flex-shrink: 0;
        }
        
        .card-type { 
            color: #666; 
            margin: 5px 0; 
            font-size: 12px;
        }
        
        .card-text { 
            margin: 10px 0; 
            line-height: 1.3; 
            font-size: 11px;
            color: #555;
            max-height: 60px;
            overflow: hidden;
        }
        
        .card-meta {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-top: 10px;
            padding-top: 10px;
            border-top: 1px solid #eee;
        }
        
        .colors { 
            font-size: 10px;
            color: #666;
        }
        
        .rarity {
            font-size: 10px;
            padding: 2px 6px;
            border-radius: 8px;
            text-transform: capitalize;
        }
        
        .rarity.common { background: #f8f9fa; color: #6c757d; }
        .rarity.uncommon { background: #e3f2fd; color: #1976d2; }
        .rarity.rare { background: #fff3e0; color: #f57c00; }
        .rarity.mythic { background: #fce4ec; color: #c2185b; }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 40px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 15px; 
            border-radius: 8px; 
            margin: 20px 0;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 100%;
            height: 100%;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 12px;
            text-align: center;
        }
        
        /* Alternative card layout for better image display */
        .card-alt {
            display: flex;
            flex-direction: column;
            height: 400px; /* Fixed height for consistency */
        }
        
        .card-alt .card-image-container {
            height: 200px; /* More space for the image */
        }
        
        .card-alt .card-content {
            flex-grow: 1;
            display: flex;
            flex-direction: column;
        }
        
        .card-alt .card-text {
            flex-grow: 1;
            overflow-y: auto;
            max-height: none;
        }
        
        @media (max-width: 768px) {
            .commander-info {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 150px;
                height: 210px;
            }
            
            .search-box input {
                width: 250px;
                margin-bottom: 10px;
            }
            
            .recommendations {
                grid-template-columns: repeat(auto-fill, minmax(250px, 1fr));
            }
        }
        
        /* Dark theme for cards */
        .card.mtg-style {
            background: linear-gradient(145deg, #2a2a2a, #1a1a1a);
            color: #e0e0e0;
            border: 1px solid #444;
        }
        
        .card.mtg-style .card-name {
            color: #fff;
        }
        
        .card.mtg-style .card-type {
            color: #ccc;
        }
        
        .card.mtg-style .card-text {
            color: #ddd;
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-info">
                    <div class="commander-image">
                        ${data.commander.image_url ? 
                            `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                            `<div class="image-placeholder" style="width: 180px; height: 251px;"></div>`
                        }
                    </div>
                    <div class="commander-details">
                        <h2>${data.commander.name}</h2>
                        <p><strong>Type:</strong> ${data.commander.type}</p>
                        <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                        <p><strong>Oracle Text:</strong> ${data.commander.text}</p>
                        ${data.commander.scryfall_url ? `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                    </div>
                </div>
                
                <h3>🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                <div class="recommendations">
            `;
            
            data.recommendations.forEach((card, index) => {
                const imageHtml = card.image_url ? 
                    `<img src="${card.image_url}" alt="${card.name}" class="card-image" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                    `<div class="image-placeholder"></div>`;
                
                html += `
                    <div class="card card-alt">
                        <div class="card-image-container">
                            ${imageHtml}
                        </div>
                        <div class="card-content">
                            <div class="card-name">
                                <div class="card-name-text">${card.name}</div>
                                <span class="similarity">${(card.similarity * 100).toFixed(1)}%</span>
                            </div>
                            <div class="card-type">${card.type}</div>
                            <div class="card-text">${card.text}</div>
                            <div class="card-meta">
                                <div class="colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                                <div class="rarity ${card.rarity}">${card.rarity}</div>
                            </div>
                            ${card.scryfall_url ? `<p style="margin: 8px 0 0 0; font-size: 11px;"><a href="${card.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                        </div>
                    </div>
                `;
            });
            
            html += '</div>';
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the fixed HTML template
with open('templates/index.html', 'w') as f:
    f.write(fixed_html_template)

print("✅ Fixed HTML template with proper image sizing!")
print("\nKey changes made:")
print("- Fixed card image aspect ratio (MTG cards are ~1.4:1 ratio)")
print("- Used object-fit: contain to show full card images")
print("- Adjusted grid layout for better card display")
print("- Added proper image container heights")
print("- Improved responsive design")
print("\nRestart your Flask app and test again!")

✅ Fixed HTML template with proper image sizing!

Key changes made:
- Fixed card image aspect ratio (MTG cards are ~1.4:1 ratio)
- Used object-fit: contain to show full card images
- Adjusted grid layout for better card display
- Added proper image container heights
- Improved responsive design

Restart your Flask app and test again!


In [14]:
# Create improved HTML template with better card container formatting
improved_html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        body { 
            font-family: Arial, sans-serif; 
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #f5f5f5;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { margin: 0; font-size: 2.5em; }
        .header p { margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9; }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 12px 15px; 
            width: 350px; 
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 25px;
            margin-right: 10px;
        }
        
        .search-box button { 
            padding: 12px 25px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 25px;
            cursor: pointer; 
            transition: transform 0.2s;
        }
        
        .search-box button:hover { transform: translateY(-2px); }
        
        .commander-info { 
            background: white;
            padding: 20px; 
            border-radius: 10px; 
            margin-bottom: 25px; 
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            display: flex;
            gap: 20px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
        }
        
        .commander-image img {
            width: 180px;
            height: 251px;
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
            background: #f8f8f8;
        }
        
        .commander-details {
            flex-grow: 1;
        }
        
        .commander-details h2 {
            margin: 0 0 10px 0;
            color: #333;
            font-size: 1.8em;
        }
        
        .recommendations { 
            display: grid; 
            grid-template-columns: repeat(auto-fill, minmax(300px, 1fr));
            gap: 20px; 
        }
        
        .card { 
            background: white;
            border: 1px solid #ddd; 
            border-radius: 12px; 
            overflow: hidden;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            transition: transform 0.2s, box-shadow 0.2s;
            display: flex;
            flex-direction: column;
            height: 480px; /* Fixed total height */
        }
        
        .card:hover {
            transform: translateY(-3px);
            box-shadow: 0 8px 15px rgba(0,0,0,0.15);
        }
        
        .card-image-container {
            width: 100%;
            height: 220px; /* Fixed height for image area */
            background: #f8f8f8;
            display: flex;
            align-items: center;
            justify-content: center;
            overflow: hidden;
        }
        
        .card-image {
            max-width: 100%;
            max-height: 100%;
            object-fit: contain;
            transition: transform 0.3s;
        }
        
        .card:hover .card-image {
            transform: scale(1.02);
        }
        
        .card-content {
            padding: 15px;
            display: flex;
            flex-direction: column;
            flex-grow: 1;
            min-height: 0; /* Important for flex layout */
        }
        
        .card-header {
            margin-bottom: 10px;
        }
        
        .card-name { 
            font-weight: bold; 
            font-size: 14px; 
            color: #333;
            margin-bottom: 5px;
            line-height: 1.3;
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            gap: 8px;
        }
        
        .card-name-text {
            flex-grow: 1;
            word-wrap: break-word;
        }
        
        .similarity { 
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white; 
            padding: 3px 7px; 
            border-radius: 10px; 
            font-size: 10px;
            font-weight: bold;
            white-space: nowrap;
            flex-shrink: 0;
        }
        
        .card-type { 
            color: #666; 
            font-size: 12px;
            margin-bottom: 8px;
        }
        
        .card-text { 
            font-size: 11px;
            color: #555;
            line-height: 1.4;
            flex-grow: 1;
            overflow-y: auto;
            margin-bottom: 10px;
        }
        
        .card-footer {
            margin-top: auto;
            padding-top: 10px;
            border-top: 1px solid #eee;
        }
        
        .card-meta {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 8px;
        }
        
        .colors { 
            font-size: 10px;
            color: #666;
        }
        
        .rarity {
            font-size: 10px;
            padding: 2px 6px;
            border-radius: 6px;
            text-transform: capitalize;
            font-weight: 500;
        }
        
        .rarity.common { background: #f8f9fa; color: #6c757d; }
        .rarity.uncommon { background: #e3f2fd; color: #1976d2; }
        .rarity.rare { background: #fff3e0; color: #f57c00; }
        .rarity.mythic { background: #fce4ec; color: #c2185b; }
        
        .scryfall-link {
            font-size: 10px;
            text-align: center;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
        }
        
        .scryfall-link a:hover {
            text-decoration: underline;
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 40px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 15px; 
            border-radius: 8px; 
            margin: 20px 0;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 150px;
            height: 210px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 12px;
            text-align: center;
            border-radius: 8px;
            margin: auto;
        }
        
        @media (max-width: 768px) {
            .commander-info {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 150px;
                height: 210px;
            }
            
            .search-box input {
                width: 250px;
                margin-bottom: 10px;
            }
            
            .recommendations {
                grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
            }
            
            .card {
                height: auto;
                min-height: 400px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-info">
                    <div class="commander-image">
                        ${data.commander.image_url ? 
                            `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                            `<div class="image-placeholder"></div>`
                        }
                    </div>
                    <div class="commander-details">
                        <h2>${data.commander.name}</h2>
                        <p><strong>Type:</strong> ${data.commander.type}</p>
                        <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                        <p><strong>Oracle Text:</strong> ${data.commander.text}</p>
                        ${data.commander.scryfall_url ? `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                    </div>
                </div>
                
                <h3>🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                <div class="recommendations">
            `;
            
            data.recommendations.forEach((card, index) => {
                const imageHtml = card.image_url ? 
                    `<img src="${card.image_url}" alt="${card.name}" class="card-image" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                    `<div class="image-placeholder"></div>`;
                
                html += `
                    <div class="card">
                        <div class="card-image-container">
                            ${imageHtml}
                        </div>
                        <div class="card-content">
                            <div class="card-header">
                                <div class="card-name">
                                    <div class="card-name-text">${card.name}</div>
                                    <span class="similarity">${(card.similarity * 100).toFixed(1)}%</span>
                                </div>
                                <div class="card-type">${card.type}</div>
                            </div>
                            <div class="card-text">${card.text}</div>
                            <div class="card-footer">
                                <div class="card-meta">
                                    <div class="colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                                    <div class="rarity ${card.rarity}">${card.rarity}</div>
                                </div>
                                ${card.scryfall_url ? `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">View on Scryfall →</a></div>` : ''}
                            </div>
                        </div>
                    </div>
                `;
            });
            
            html += '</div>';
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the improved HTML template
with open('templates/index.html', 'w') as f:
    f.write(improved_html_template)

print("✅ Improved HTML template with better card container layout!")
print("\nKey improvements:")
print("- Fixed card container structure with proper flex layout")
print("- Consistent card heights (480px) for uniform grid")
print("- Better image container centering and sizing")
print("- Improved content organization (header, body, footer)")
print("- Better responsive design for mobile")
print("- Proper text overflow handling")
print("\nRestart your Flask app to see the improvements!")

✅ Improved HTML template with better card container layout!

Key improvements:
- Fixed card container structure with proper flex layout
- Consistent card heights (480px) for uniform grid
- Better image container centering and sizing
- Improved content organization (header, body, footer)
- Better responsive design for mobile
- Proper text overflow handling

Restart your Flask app to see the improvements!


In [15]:
# Create a completely fixed HTML template with proper card layout
fixed_layout_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #f5f5f5;
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        .cards-grid { 
            display: grid; 
            grid-template-columns: repeat(auto-fit, minmax(320px, 1fr));
            gap: 25px; 
            padding: 0;
        }
        
        .recommendation-card { 
            background: white;
            border-radius: 15px; 
            overflow: hidden;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
            border: 1px solid #e0e0e0;
        }
        
        .recommendation-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 25px rgba(0,0,0,0.15);
        }
        
        .card-image-section {
            height: 240px;
            background: #f8f8f8;
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 15px;
        }
        
        .card-image-section img {
            max-width: 100%;
            max-height: 100%;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .card-details {
            padding: 20px;
        }
        
        .card-name-row {
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            margin-bottom: 10px;
            gap: 10px;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            line-height: 1.3;
            flex-grow: 1;
        }
        
        .similarity-badge { 
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white; 
            padding: 6px 12px; 
            border-radius: 20px; 
            font-size: 12px;
            font-weight: 700;
            white-space: nowrap;
            flex-shrink: 0;
        }
        
        .card-type { 
            color: #666; 
            font-size: 14px;
            margin-bottom: 12px;
            font-weight: 500;
        }
        
        .card-text { 
            font-size: 13px;
            color: #555;
            line-height: 1.5;
            margin-bottom: 15px;
            min-height: 60px;
        }
        
        .card-meta-row {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding-top: 15px;
            border-top: 1px solid #f0f0f0;
            margin-bottom: 10px;
        }
        
        .card-colors { 
            font-size: 12px;
            color: #666;
            font-weight: 500;
        }
        
        .rarity-badge {
            font-size: 11px;
            padding: 4px 8px;
            border-radius: 12px;
            text-transform: capitalize;
            font-weight: 600;
        }
        
        .rarity-badge.common { background: #f8f9fa; color: #6c757d; }
        .rarity-badge.uncommon { background: #e3f2fd; color: #1976d2; }
        .rarity-badge.rare { background: #fff3e0; color: #f57c00; }
        .rarity-badge.mythic { background: #fce4ec; color: #c2185b; }
        
        .scryfall-link {
            text-align: center;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 12px;
            font-weight: 500;
        }
        
        .scryfall-link a:hover {
            text-decoration: underline;
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 180px;
            height: 252px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .cards-grid {
                grid-template-columns: 1fr;
                gap: 20px;
            }
            
            .header h1 {
                font-size: 2em;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">
                            ${data.commander.image_url ? 
                                `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                                `<div class="image-placeholder"></div>`
                            }
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>
                            ${data.commander.scryfall_url ? `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>` : ''}
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="cards-grid">
            `;
            
            data.recommendations.forEach((card, index) => {
                const imageHtml = card.image_url ? 
                    `<img src="${card.image_url}" alt="${card.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>'">` :
                    `<div class="image-placeholder"></div>`;
                
                html += `
                    <div class="recommendation-card">
                        <div class="card-image-section">
                            ${imageHtml}
                        </div>
                        <div class="card-details">
                            <div class="card-name-row">
                                <div class="card-name">${card.name}</div>
                                <span class="similarity-badge">${(card.similarity * 100).toFixed(1)}%</span>
                            </div>
                            <div class="card-type">${card.type}</div>
                            <div class="card-text">${card.text}</div>
                            <div class="card-meta-row">
                                <div class="card-colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                                <div class="rarity-badge ${card.rarity}">${card.rarity}</div>
                            </div>
                            ${card.scryfall_url ? `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">View on Scryfall →</a></div>` : ''}
                        </div>
                    </div>
                `;
            });
            
            html += `
                    </div>
                </div>
            `;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the completely fixed template
with open('templates/index.html', 'w') as f:
    f.write(fixed_layout_template)

print("✅ Completely restructured HTML template with proper layout!")
print("\nKey fixes applied:")
print("- Added CSS reset to prevent box model issues")
print("- Simplified grid layout with proper sizing")
print("- Fixed card structure with clear sections")
print("- Proper image containers with fixed dimensions")
print("- Better responsive design")
print("- Cleaner spacing and typography")
print("\nRestart your Flask app - this should fix all the layout issues!")

✅ Completely restructured HTML template with proper layout!

Key fixes applied:
- Added CSS reset to prevent box model issues
- Simplified grid layout with proper sizing
- Fixed card structure with clear sections
- Proper image containers with fixed dimensions
- Better responsive design
- Cleaner spacing and typography

Restart your Flask app - this should fix all the layout issues!


In [19]:
# Create properly sized HTML template without the stray text
properly_sized_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #f5f5f5;
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* Fixed grid with proper card sizing */
        .cards-grid { 
            display: grid; 
            grid-template-columns: repeat(auto-fill, minmax(280px, 1fr));
            gap: 20px; 
            padding: 0;
            max-width: 100%;
        }
        
        .recommendation-card { 
            background: white;
            border-radius: 15px; 
            overflow: hidden;
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
            transition: all 0.3s ease;
            border: 1px solid #e0e0e0;
            width: 100%;
            max-width: 320px; /* Prevent cards from getting too wide */
            margin: 0 auto; /* Center cards in their grid area */
        }
        
        .recommendation-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 25px rgba(0,0,0,0.15);
        }
        
        .card-image-section {
            height: 200px;
            background: #f8f8f8;
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 10px;
        }
        
        .card-image-section img {
            max-width: 100%;
            max-height: 100%;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .card-details {
            padding: 16px;
        }
        
        .card-name-row {
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            margin-bottom: 8px;
            gap: 8px;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 14px;
            color: #333;
            line-height: 1.3;
            flex-grow: 1;
        }
        
        .similarity-badge { 
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white; 
            padding: 4px 8px; 
            border-radius: 15px; 
            font-size: 11px;
            font-weight: 700;
            white-space: nowrap;
            flex-shrink: 0;
        }
        
        .card-type { 
            color: #666; 
            font-size: 12px;
            margin-bottom: 10px;
            font-weight: 500;
        }
        
        .card-text { 
            font-size: 11px;
            color: #555;
            line-height: 1.4;
            margin-bottom: 12px;
            min-height: 40px;
            max-height: 80px;
            overflow: hidden;
        }
        
        .card-meta-row {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding-top: 10px;
            border-top: 1px solid #f0f0f0;
            margin-bottom: 8px;
        }
        
        .card-colors { 
            font-size: 10px;
            color: #666;
            font-weight: 500;
        }
        
        .rarity-badge {
            font-size: 9px;
            padding: 3px 6px;
            border-radius: 10px;
            text-transform: capitalize;
            font-weight: 600;
        }
        
        .rarity-badge.common { background: #f8f9fa; color: #6c757d; }
        .rarity-badge.uncommon { background: #e3f2fd; color: #1976d2; }
        .rarity-badge.rare { background: #fff3e0; color: #f57c00; }
        .rarity-badge.mythic { background: #fce4ec; color: #c2185b; }
        
        .scryfall-link {
            text-align: center;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 10px;
            font-weight: 500;
        }
        
        .scryfall-link a:hover {
            text-decoration: underline;
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 150px;
            height: 210px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 12px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
            margin: 0 auto;
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .cards-grid {
                grid-template-columns: 1fr;
                gap: 15px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .recommendation-card {
                max-width: 100%;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>`;
            } else {
                html += `<div class="image-placeholder"></div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="cards-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="recommendation-card">
                        <div class="card-image-section">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\"></div>`;
                } else {
                    html += `<div class="image-placeholder"></div>`;
                }
                
                html += `
                        </div>
                        <div class="card-details">
                            <div class="card-name-row">
                                <div class="card-name">${card.name}</div>
                                <span class="similarity-badge">${(card.similarity * 100).toFixed(1)}%</span>
                            </div>
                            <div class="card-type">${card.type}</div>
                            <div class="card-text">${card.text}</div>
                            <div class="card-meta-row">
                                <div class="card-colors">Colors: ${card.colors.join(', ') || 'Colorless'}</div>
                                <div class="rarity-badge ${card.rarity}">${card.rarity}</div>
                            </div>`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">View on Scryfall →</a></div>`;
                }
                
                html += `
                        </div>
                    </div>`;
            });
            
            html += `
                    </div>
                </div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the properly sized template
with open('templates/index.html', 'w') as f:
    f.write(properly_sized_template)

print("✅ Fixed card sizing and cleaned up HTML!")
print("\nKey fixes:")
print("- Cards now have max-width: 320px to prevent excessive width")
print("- Grid uses minmax(280px, 1fr) for better sizing")
print("- Removed the stray text by properly structuring HTML strings")
print("- Cards are centered in their grid areas")
print("- Better responsive design for mobile")
print("\nRestart your Flask app - cards should now be properly sized!")

✅ Fixed card sizing and cleaned up HTML!

Key fixes:
- Cards now have max-width: 320px to prevent excessive width
- Grid uses minmax(280px, 1fr) for better sizing
- Removed the stray text by properly structuring HTML strings
- Cards are centered in their grid areas
- Better responsive design for mobile

Restart your Flask app - cards should now be properly sized!


In [20]:
# Create the new layout template with your specifications
three_column_results_layout = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3; /* Light gray background */
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* 3-column grid layout */
        .results-grid { 
            display: grid; 
            grid-template-columns: repeat(3, 1fr);
            gap: 20px; 
            padding: 0;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080; /* Medium gray outline */
            border-radius: 10px; 
            padding: 20px;
            text-align: center; /* Middle aligned content */
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
        }
        
        .result-box:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .similarity-score {
            position: absolute;
            top: 15px;
            right: 15px;
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 6px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 700;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 18px;
            color: #333;
            margin-bottom: 10px;
            margin-top: 0;
            line-height: 1.3;
        }
        
        .card-meta {
            font-size: 14px;
            color: #666;
            margin-bottom: 8px;
            font-weight: 500;
        }
        
        .card-type {
            font-size: 14px;
            color: #555;
            margin-bottom: 12px;
            font-weight: 500;
        }
        
        .card-oracle-text {
            font-size: 13px;
            color: #444;
            line-height: 1.4;
            margin-bottom: 15px;
            text-align: left; /* Oracle text reads better left-aligned */
            background: #f9f9f9;
            padding: 10px;
            border-radius: 6px;
            border-left: 3px solid #667eea;
        }
        
        .card-image-container {
            margin: 15px 0;
            display: flex;
            justify-content: center;
        }
        
        .card-image-container img {
            width: 225px; /* 50% larger than previous 150px */
            height: 315px; /* 50% larger, maintaining MTG card ratio */
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 3px 6px rgba(0,0,0,0.2);
            background: white;
        }
        
        .scryfall-link {
            margin-top: 15px;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 14px;
            font-weight: 600;
            padding: 8px 16px;
            border: 2px solid #667eea;
            border-radius: 20px;
            transition: all 0.3s ease;
            display: inline-block;
        }
        
        .scryfall-link a:hover {
            background: #667eea;
            color: white;
            transform: translateY(-2px);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 225px;
            height: 315px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
            margin: 0 auto;
        }
        
        /* Responsive design */
        @media (max-width: 1200px) {
            .results-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .results-grid {
                grid-template-columns: 1fr;
                gap: 15px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .card-image-container img {
                width: 180px;
                height: 252px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\">Image not available</div>'">`;
            } else {
                html += `<div class="image-placeholder">Image not available</div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="results-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="result-box">
                        <div class="similarity-score">${(card.similarity * 100).toFixed(1)}%</div>
                        
                        <div class="card-name">${card.name}</div>
                        
                        <div class="card-meta">
                            Colors: ${card.colors.join(', ') || 'Colorless'} | 
                            CMC: ${card.mana_cost || 'N/A'} | 
                            ${card.rarity}
                        </div>
                        
                        <div class="card-type">${card.type}</div>
                        
                        <div class="card-oracle-text">${card.text}</div>
                        
                        <div class="card-image-container">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\">Image not available</div>'">`;
                } else {
                    html += `<div class="image-placeholder">Image not available</div>`;
                }
                
                html += `
                        </div>`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">View on Scryfall</a></div>`;
                }
                
                html += `
                    </div>`;
            });
            
            html += `
                    </div>
                </div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the new layout template
with open('templates/index.html', 'w') as f:
    f.write(three_column_results_layout)

print("✅ Created new 3-column results layout!")
print("\nNew features:")
print("- Light gray background (#d3d3d3)")
print("- 3 result boxes per row")
print("- White boxes with medium gray outlines")
print("- Similarity score in top right corner")
print("- Card name in bold at top (center aligned)")
print("- Colors, CMC, and rarity on one line")
print("- Card type below that")
print("- Oracle text in styled box")
print("- Card images 50% larger (225x315px)")
print("- Scryfall link as styled button")
print("- Responsive design (2 columns on medium screens, 1 on mobile)")
print("\nRestart your Flask app to see the new layout!")

✅ Created new 3-column results layout!

New features:
- Light gray background (#d3d3d3)
- 3 result boxes per row
- White boxes with medium gray outlines
- Similarity score in top right corner
- Card name in bold at top (center aligned)
- Colors, CMC, and rarity on one line
- Card type below that
- Oracle text in styled box
- Card images 50% larger (225x315px)
- Scryfall link as styled button
- Responsive design (2 columns on medium screens, 1 on mobile)

Restart your Flask app to see the new layout!


In [24]:
# Create the corrected layout with proper 3-column grid
corrected_three_column_layout = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3; /* Light gray background */
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* Fixed 3-column grid layout */
        .results-grid { 
            display: grid; 
            grid-template-columns: 1fr 1fr 1fr; /* Force exactly 3 equal columns */
            gap: 20px; 
            padding: 0;
            width: 100%;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080; /* Medium gray outline */
            border-radius: 10px; 
            padding: 20px;
            text-align: center; /* Middle aligned content */
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
            min-height: 500px; /* Ensure consistent height */
            display: flex;
            flex-direction: column;
        }
        
        .result-box:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .similarity-score {
            position: absolute;
            top: 15px;
            right: 15px;
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 6px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 700;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 18px;
            color: #333;
            margin-bottom: 10px;
            margin-top: 0;
            line-height: 1.3;
        }
        
        .card-meta {
            font-size: 14px;
            color: #666;
            margin-bottom: 8px;
            font-weight: 500;
        }
        
        .card-type {
            font-size: 14px;
            color: #555;
            margin-bottom: 12px;
            font-weight: 500;
        }
        
        .card-oracle-text {
            font-size: 13px;
            color: #444;
            line-height: 1.4;
            margin-bottom: 15px;
            text-align: left; /* Oracle text reads better left-aligned */
            background: #f9f9f9;
            padding: 10px;
            border-radius: 6px;
            border-left: 3px solid #667eea;
            flex-grow: 1; /* Take up available space */
        }
        
        .card-image-container {
            margin: 15px 0;
            display: flex;
            justify-content: center;
        }
        
        .card-image-container img {
            width: 200px; /* 33% bigger than previous 150px (150 * 1.33 = 200) */
            height: 280px; /* 33% bigger, maintaining MTG card ratio */
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 3px 6px rgba(0,0,0,0.2);
            background: white;
        }
        
        .card-footer {
            margin-top: auto; /* Push to bottom */
            position: relative;
            height: 40px; /* Reserve space for the link */
        }
        
        .scryfall-link {
            position: absolute;
            bottom: 0;
            left: 0;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 12px;
            font-weight: 600;
            padding: 6px 12px;
            border: 1px solid #667eea;
            border-radius: 15px;
            transition: all 0.3s ease;
            display: inline-block;
        }
        
        .scryfall-link a:hover {
            background: #667eea;
            color: white;
            transform: translateY(-1px);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 200px;
            height: 280px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
            margin: 0 auto;
        }
        
        /* Responsive design */
        @media (max-width: 1200px) {
            .results-grid {
                grid-template-columns: 1fr 1fr; /* 2 columns on medium screens */
            }
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .results-grid {
                grid-template-columns: 1fr; /* 1 column on mobile */
                gap: 15px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .card-image-container img {
                width: 160px;
                height: 224px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\">Image not available</div>'">`;
            } else {
                html += `<div class="image-placeholder">Image not available</div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="results-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="result-box">
                        <div class="similarity-score">${(card.similarity * 100).toFixed(1)}%</div>
                        
                        <div class="card-name">${card.name}</div>
                        
                        <div class="card-meta">
                            Colors: ${card.colors.join(', ') || 'Colorless'} | 
                            CMC: ${card.mana_cost || 'N/A'} | 
                            ${card.rarity}
                        </div>
                        
                        <div class="card-type">${card.type}</div>
                        
                        <div class="card-oracle-text">${card.text}</div>
                        
                        <div class="card-image-container">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}" onerror="this.parentElement.innerHTML='<div class=\\"image-placeholder\\">Image not available</div>'">`;
                } else {
                    html += `<div class="image-placeholder">Image not available</div>`;
                }
                
                html += `
                        </div>
                        
                        <div class="card-footer">`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">Scryfall</a></div>`;
                }
                
                html += `
                        </div>
                    </div>`;
            });
            
            html += `
                    </div>
                </div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the corrected layout template
with open('templates/index.html', 'w') as f:
    f.write(corrected_three_column_layout)

print("✅ Fixed 3-column layout with adjustments!")
print("\nKey fixes:")
print("- Force exactly 3 columns: grid-template-columns: 1fr 1fr 1fr")
print("- Moved Scryfall link to bottom left corner of each box")
print("- Images now 33% bigger (200x280px instead of 150x210px)")
print("- Added flex layout to ensure proper spacing")
print("- Consistent box heights with proper footer positioning")
print("\nRestart your Flask app - you should now see exactly 3 boxes per row!")

✅ Fixed 3-column layout with adjustments!

Key fixes:
- Force exactly 3 columns: grid-template-columns: 1fr 1fr 1fr
- Moved Scryfall link to bottom left corner of each box
- Images now 33% bigger (200x280px instead of 150x210px)
- Added flex layout to ensure proper spacing
- Consistent box heights with proper footer positioning

Restart your Flask app - you should now see exactly 3 boxes per row!


In [25]:
# Create the fixed layout without the display issues
fixed_layout_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3; /* Light gray background */
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* Fixed 3-column grid layout with consistent sizing */
        .results-grid { 
            display: grid; 
            grid-template-columns: repeat(3, 1fr);
            gap: 20px; 
            padding: 0;
            width: 100%;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080; /* Medium gray outline */
            border-radius: 10px; 
            padding: 20px;
            text-align: center;
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
            height: 650px; /* Fixed height for all boxes */
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }
        
        .result-box:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .similarity-score {
            position: absolute;
            top: 15px;
            right: 15px;
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 6px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 700;
            z-index: 10;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            margin-bottom: 8px;
            margin-top: 0;
            line-height: 1.3;
            padding-right: 60px; /* Space for similarity score */
        }
        
        .card-meta {
            font-size: 12px;
            color: #666;
            margin-bottom: 6px;
            font-weight: 500;
        }
        
        .card-type {
            font-size: 12px;
            color: #555;
            margin-bottom: 8px;
            font-weight: 500;
        }
        
        .card-oracle-text {
            font-size: 11px;
            color: #444;
            line-height: 1.3;
            margin-bottom: 10px;
            text-align: left;
            background: #f9f9f9;
            padding: 8px;
            border-radius: 6px;
            border-left: 3px solid #667eea;
            max-height: 80px;
            overflow-y: auto;
        }
        
        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center;
            flex-grow: 1;
            align-items: center;
        }
        
        .card-image-container img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 3px 6px rgba(0,0,0,0.2);
            background: white;
        }
        
        .card-footer {
            margin-top: auto;
            position: relative;
            height: 30px;
            width: 100%;
        }
        
        .scryfall-link {
            position: absolute;
            bottom: 0;
            left: 0;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 11px;
            font-weight: 600;
            padding: 4px 8px;
            border: 1px solid #667eea;
            border-radius: 12px;
            transition: all 0.3s ease;
            display: inline-block;
        }
        
        .scryfall-link a:hover {
            background: #667eea;
            color: white;
            transform: translateY(-1px);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 200px;
            height: 280px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
        }
        
        /* Responsive design */
        @media (max-width: 1200px) {
            .results-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .results-grid {
                grid-template-columns: 1fr;
                gap: 15px;
            }
            
            .result-box {
                height: auto;
                min-height: 500px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .card-image-container img {
                width: 160px;
                height: 224px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 12 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}">`;
            } else {
                html += `<div class="image-placeholder">Image not available</div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="results-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="result-box">
                        <div class="similarity-score">${(card.similarity * 100).toFixed(1)}%</div>
                        
                        <div class="card-name">${card.name}</div>
                        
                        <div class="card-meta">Colors: ${card.colors.join(', ') || 'Colorless'} | CMC: ${card.mana_cost || 'N/A'} | ${card.rarity}</div>
                        
                        <div class="card-type">${card.type}</div>
                        
                        <div class="card-oracle-text">${card.text}</div>
                        
                        <div class="card-image-container">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}">`;
                } else {
                    html += `<div class="image-placeholder">Image not available</div>`;
                }
                
                html += `</div>
                        
                        <div class="card-footer">`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">Scryfall</a></div>`;
                }
                
                html += `</div>
                    </div>`;
            });
            
            html += `</div></div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the fixed layout template
with open('templates/index.html', 'w') as f:
    f.write(fixed_layout_template)

print("✅ Fixed all display issues!")
print("\nKey fixes:")
print("- Fixed box sizing: all boxes now have consistent height (650px)")
print("- Removed stray '\">' text by cleaning up HTML string concatenation")
print("- Fixed image error handling - only shows 'Image not available' when actually needed")
print("- Consistent 3-column layout with proper spacing")
print("- Better overflow handling for long text")
print("\nRestart your Flask app - the layout should now be clean and consistent!")

✅ Fixed all display issues!

Key fixes:
- Fixed box sizing: all boxes now have consistent height (650px)
- Removed stray '">' text by cleaning up HTML string concatenation
- Fixed image error handling - only shows 'Image not available' when actually needed
- Consistent 3-column layout with proper spacing
- Better overflow handling for long text

Restart your Flask app - the layout should now be clean and consistent!


In [26]:
# Read the current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Update just the image dimensions
content = content.replace('width: 200px;', 'width: 266px;')
content = content.replace('height: 280px;', 'height: 372px;')

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated image sizes to be 33% larger!")
print("Images are now 266px x 372px (was 200px x 280px)")
print("Refresh your browser to see the larger images!")

✅ Updated image sizes to be 33% larger!
Images are now 266px x 372px (was 200px x 280px)
Refresh your browser to see the larger images!


In [27]:
# Read current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Add mana symbol CSS
mana_css = '''
        /* Mana symbol styles */
        .mana-symbol {
            display: inline-block;
            width: 18px;
            height: 18px;
            border-radius: 50%;
            margin: 0 2px;
            text-align: center;
            line-height: 18px;
            font-size: 11px;
            font-weight: bold;
            vertical-align: middle;
            border: 1px solid rgba(0,0,0,0.2);
            box-shadow: 0 1px 2px rgba(0,0,0,0.1);
        }
        
        .mana-w { background: linear-gradient(135deg, #fffbd5, #f8f4c0); color: #333; }
        .mana-u { background: linear-gradient(135deg, #0e68ab, #1565c0); color: white; }
        .mana-b { background: linear-gradient(135deg, #2e2e2e, #1a1a1a); color: white; }
        .mana-r { background: linear-gradient(135deg, #d32f2f, #c62828); color: white; }
        .mana-g { background: linear-gradient(135deg, #388e3c, #2e7d32); color: white; }
        .mana-c { background: linear-gradient(135deg, #9e9e9e, #757575); color: white; }
        .mana-generic { background: linear-gradient(135deg, #e0e0e0, #bdbdbd); color: #333; }
        
        .mana-cost-display {
            display: inline-flex;
            align-items: center;
            gap: 1px;
        }'''

# Insert CSS
content = content.replace('        @media (max-width: 1200px)', mana_css + '\n\n        @media (max-width: 1200px)')

# Add JavaScript function
mana_js = '''
        
        function createManaSymbols(manaString) {
            if (!manaString || manaString === 'N/A' || manaString === '') {
                return '<span style="color: #999; font-size: 12px;">Free</span>';
            }
            
            const symbols = manaString.match(/\\{[^}]+\\}/g) || [];
            if (symbols.length === 0) return manaString;
            
            const manaHtml = symbols.map(symbol => {
                const clean = symbol.replace(/[{}]/g, '');
                
                if (clean === 'W') return '<span class="mana-symbol mana-w">W</span>';
                if (clean === 'U') return '<span class="mana-symbol mana-u">U</span>';
                if (clean === 'B') return '<span class="mana-symbol mana-b">B</span>';
                if (clean === 'R') return '<span class="mana-symbol mana-r">R</span>';
                if (clean === 'G') return '<span class="mana-symbol mana-g">G</span>';
                if (clean === 'C') return '<span class="mana-symbol mana-c">C</span>';
                if (clean === 'X') return '<span class="mana-symbol mana-x">X</span>';
                if (/^\\d+$/.test(clean)) return `<span class="mana-symbol mana-generic">${clean}</span>`;
                
                return `<span class="mana-symbol mana-generic">${clean}</span>`;
            }).join('');
            
            return `<span class="mana-cost-display">${manaHtml}</span>`;
        }'''

# Insert JavaScript
content = content.replace('        // Allow Enter key to trigger search', mana_js + '\n\n        // Allow Enter key to trigger search')

# Update the card meta display
old_line = '<div class="card-meta">Colors: ${card.colors.join(\', \') || \'Colorless\'} | CMC: ${card.mana_cost || \'N/A\'} | ${card.rarity}</div>'
new_line = '<div class="card-meta">Colors: ${card.colors.join(\', \') || \'Colorless\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity}</div>'

content = content.replace(old_line, new_line)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Added mana symbol support!")
print("Changes made:")
print("- Added CSS for colorful mana symbols")
print("- Added JavaScript function to parse {1}{W}{U} format")
print("- Updated card display to show mana symbols instead of text")
print("- Symbols are color-coded: W=white, U=blue, B=black, R=red, G=green")
print("\nRestart your Flask app and search for a commander to see the mana symbols!")

✅ Added mana symbol support!
Changes made:
- Added CSS for colorful mana symbols
- Added JavaScript function to parse {1}{W}{U} format
- Updated card display to show mana symbols instead of text
- Symbols are color-coded: W=white, U=blue, B=black, R=red, G=green

Restart your Flask app and search for a commander to see the mana symbols!


In [28]:
# Read the current app.py
with open('app.py', 'r') as f:
    app_content = f.read()

# Add a new route to get all commanders
commander_route = '''
@app.route('/commanders')
def get_commanders():
    """Get list of all available commanders"""
    try:
        # Find all legendary creatures (potential commanders)
        commanders = df_clean[
            df_clean['type_line'].str.contains('Legendary Creature', case=False, na=False)
        ].copy()
        
        # Sort by name for easier browsing
        commanders = commanders.sort_values('name')
        
        commander_list = []
        for _, row in commanders.iterrows():
            commander_list.append({
                'name': row['name'],
                'colors': list(get_color_identity(row['colors'])),
                'type': row['type_line']
            })
        
        return jsonify({
            'commanders': commander_list,
            'total': len(commander_list)
        })
        
    except Exception as e:
        return jsonify({'error': str(e)})
'''

# Insert the new route before the existing @app.route('/') line
app_content = app_content.replace(
    '@app.route(\'/\')',
    commander_route + '\n@app.route(\'/\')'
)

# Save the updated app
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Updated Flask app with commander list endpoint!")

✅ Updated Flask app with commander list endpoint!


In [29]:
# Read the current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Replace the search box with a dropdown version
old_search_box = '''<div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <select id="numResults" style="padding: 15px; font-size: 16px; border: 2px solid #ddd; border-radius: 30px; margin: 0 10px;">
            <option value="12">12 results</option>
            <option value="24" selected>24 results</option>
            <option value="36">36 results</option>
            <option value="48">48 results</option>
        </select>
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>'''

new_search_box = '''<div class="search-box">
        <select id="commanderSelect" style="padding: 15px 20px; width: 400px; max-width: 60%; font-size: 16px; border: 2px solid #ddd; border-radius: 30px; margin-right: 15px; background: white;">
            <option value="">Loading commanders...</option>
        </select>
        <select id="numResults" style="padding: 15px; font-size: 16px; border: 2px solid #ddd; border-radius: 30px; margin: 0 10px;">
            <option value="12">12 results</option>
            <option value="24">24 results</option>
            <option value="30" selected>30 results</option>
            <option value="36">36 results</option>
            <option value="48">48 results</option>
        </select>
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>'''

content = content.replace(old_search_box, new_search_box)

# Update the JavaScript to load commanders and use dropdown
old_js_function = '''async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }'''

new_js_function = '''async function getRecommendations() {
            const commander = document.getElementById('commanderSelect').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander) {
                resultsDiv.innerHTML = '<div class="error">Please select a commander</div>';
                return;
            }'''

content = content.replace(old_js_function, new_js_function)

# Update the num_recommendations line
content = content.replace('num_recommendations: 12', 'num_recommendations: parseInt(document.getElementById("numResults").value)')

# Add function to load commanders
load_commanders_js = '''
        
        async function loadCommanders() {
            try {
                const response = await fetch('/commanders');
                const data = await response.json();
                
                const select = document.getElementById('commanderSelect');
                select.innerHTML = '<option value="">Choose a commander...</option>';
                
                if (data.commanders) {
                    data.commanders.forEach(commander => {
                        const colorStr = commander.colors.length > 0 ? ` (${commander.colors.join('')})` : '';
                        const option = document.createElement('option');
                        option.value = commander.name;
                        option.textContent = `${commander.name}${colorStr}`;
                        select.appendChild(option);
                    });
                    
                    console.log(`Loaded ${data.total} commanders`);
                } else {
                    select.innerHTML = '<option value="">Error loading commanders</option>';
                }
                
            } catch (error) {
                console.error('Error loading commanders:', error);
                const select = document.getElementById('commanderSelect');
                select.innerHTML = '<option value="">Error loading commanders</option>';
            }
        }
        
        // Load commanders when page loads
        window.addEventListener('DOMContentLoaded', loadCommanders);'''

# Insert the new function before the existing event listener
content = content.replace(
    '        // Allow Enter key to trigger search',
    load_commanders_js + '\n\n        // Allow Enter key to trigger search'
)

# Remove the old enter key listener since we're using a dropdown now
old_listener = '''document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });'''

new_listener = '''document.getElementById('commanderSelect').addEventListener('change', function(e) {
            if (e.target.value) {
                // Auto-search when commander is selected
                // getRecommendations(); // Uncomment this line for auto-search
            }
        });'''

content = content.replace(old_listener, new_listener)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated HTML template with commander dropdown!")
print("\nChanges made:")
print("- Replaced text input with dropdown select")
print("- Added commander loading function")
print("- Updated JavaScript to use selected commander")
print("- Shows commander colors in dropdown (e.g., 'Atraxa (WUBG)')")
print("- Dropdown loads all legendary creatures from dataset")
print("\nRestart your Flask app to test the new dropdown!")

✅ Updated HTML template with commander dropdown!

Changes made:
- Replaced text input with dropdown select
- Added commander loading function
- Updated JavaScript to use selected commander
- Shows commander colors in dropdown (e.g., 'Atraxa (WUBG)')
- Dropdown loads all legendary creatures from dataset

Restart your Flask app to test the new dropdown!


In [30]:
# Add some additional CSS for the dropdown
dropdown_css = '''
        
        #commanderSelect {
            appearance: none;
            background-image: url("data:image/svg+xml;charset=UTF-8,%3csvg xmlns='http://www.w3.org/2000/svg' viewBox='0 0 24 24' fill='none' stroke='currentColor' stroke-width='2' stroke-linecap='round' stroke-linejoin='round'%3e%3cpolyline points='6,9 12,15 18,9'%3e%3c/polyline%3e%3c/svg%3e");
            background-repeat: no-repeat;
            background-position: right 12px center;
            background-size: 20px;
            cursor: pointer;
        }
        
        #commanderSelect:focus {
            border-color: #667eea;
            outline: none;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }'''

# Read template and add the CSS
with open('templates/index.html', 'r') as f:
    content = f.read()

content = content.replace(
    '        .mana-cost-display {',
    dropdown_css + '\n\n        .mana-cost-display {'
)

with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Added dropdown styling!")

✅ Added dropdown styling!


In [31]:
# First, let's create a simple test to see if the /commanders endpoint works
test_commanders_code = '''
@app.route('/test_commanders')
def test_commanders():
    """Simple test endpoint to check commanders"""
    try:
        # Check if we can find legendary creatures
        commanders = df_clean[df_clean['type_line'].str.contains('Legendary Creature', case=False, na=False)]
        count = len(commanders)
        
        # Get first 5 as examples
        examples = commanders.head(5)['name'].tolist()
        
        return jsonify({
            'status': 'success',
            'total_commanders': count,
            'examples': examples,
            'total_cards': len(df_clean)
        })
    except Exception as e:
        return jsonify({
            'status': 'error',
            'error': str(e)
        })
'''

# Add this test endpoint to app.py
with open('app.py', 'r') as f:
    app_content = f.read()

# Insert before the main route
app_content = app_content.replace(
    '@app.route(\'/\')',
    test_commanders_code + '\n@app.route(\'/\')'
)

with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Added test endpoint. Restart Flask and visit:")
print("http://localhost:5001/test_commanders")
print("This will show if commanders are being found correctly.")

✅ Added test endpoint. Restart Flask and visit:
http://localhost:5001/test_commanders
This will show if commanders are being found correctly.


In [32]:
# Create a simplified working dropdown template
simplified_dropdown_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3;
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-controls {
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 15px;
            flex-wrap: wrap;
        }
        
        #commanderSelect {
            padding: 15px 20px;
            min-width: 300px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 30px;
            background: white;
            cursor: pointer;
        }
        
        #commanderSelect:focus {
            border-color: #667eea;
            outline: none;
        }
        
        #numResults {
            padding: 15px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 30px;
            background: white;
        }
        
        .search-button {
            padding: 15px 30px;
            font-size: 16px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 30px;
            cursor: pointer;
            font-weight: 600;
        }
        
        .search-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        /* Add your existing result styles here */
        .results-grid { 
            display: grid; 
            grid-template-columns: repeat(3, 1fr);
            gap: 20px; 
            padding: 0;
            width: 100%;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080;
            border-radius: 10px; 
            padding: 20px;
            text-align: center;
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            height: 650px;
            display: flex;
            flex-direction: column;
        }
        
        /* Add all your other existing CSS here... */
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Select a commander to get AI-powered card recommendations!</p>
    </div>

    <div class="search-box">
        <div class="search-controls">
            <select id="commanderSelect">
                <option value="">Loading commanders...</option>
            </select>
            
            <select id="numResults">
                <option value="12">12 results</option>
                <option value="24">24 results</option>
                <option value="30" selected>30 results</option>
                <option value="36">36 results</option>
            </select>
            
            <button class="search-button" onclick="getRecommendations()">Get Recommendations</button>
        </div>
    </div>

    <div id="results"></div>

    <script>
        // Simple commander loading
        async function loadCommanders() {
            console.log('Loading commanders...');
            
            try {
                const response = await fetch('/test_commanders');
                const data = await response.json();
                
                console.log('Test response:', data);
                
                if (data.status === 'success') {
                    // For now, just add a few test commanders
                    const select = document.getElementById('commanderSelect');
                    select.innerHTML = `
                        <option value="">Choose a commander...</option>
                        <option value="Atraxa">Atraxa, Praetors' Voice</option>
                        <option value="Arcades">Arcades, the Strategist</option>
                        <option value="Isshin">Isshin, Two Heavens as One</option>
                        <option value="Krenko">Krenko, Mob Boss</option>
                    `;
                    console.log('Basic commanders loaded');
                } else {
                    throw new Error(data.error || 'Unknown error');
                }
                
            } catch (error) {
                console.error('Error loading commanders:', error);
                const select = document.getElementById('commanderSelect');
                select.innerHTML = '<option value="">Error loading commanders</option>';
            }
        }
        
        // Simple recommendation function
        function getRecommendations() {
            console.log('Get recommendations clicked');
            
            const commander = document.getElementById('commanderSelect').value;
            const numResults = document.getElementById('numResults').value;
            const resultsDiv = document.getElementById('results');
            
            console.log('Selected commander:', commander);
            console.log('Number of results:', numResults);
            
            if (!commander) {
                resultsDiv.innerHTML = '<div class="error">Please select a commander</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            // For now, just show this works
            setTimeout(() => {
                resultsDiv.innerHTML = `<div class="loading">Testing with commander: ${commander} (${numResults} results)</div>`;
            }, 1000);
        }
        
        // Load commanders when page loads
        window.addEventListener('DOMContentLoaded', function() {
            console.log('Page loaded, loading commanders...');
            loadCommanders();
        });
    </script>
</body>
</html>
'''

# Save this simplified version to test
with open('templates/index_test.html', 'w') as f:
    f.write(simplified_dropdown_template)

print("✅ Created simplified test template!")
print("\nTo test:")
print("1. Restart your Flask app")
print("2. Visit http://localhost:5001/test_commanders to see if commanders are found")
print("3. Temporarily rename templates/index.html to templates/index_backup.html")
print("4. Rename templates/index_test.html to templates/index.html")
print("5. Test the basic dropdown functionality")

✅ Created simplified test template!

To test:
1. Restart your Flask app
2. Visit http://localhost:5001/test_commanders to see if commanders are found
3. Temporarily rename templates/index.html to templates/index_backup.html
4. Rename templates/index_test.html to templates/index.html
5. Test the basic dropdown functionality


In [33]:
# Remove the commander routes we added to app.py
with open('app.py', 'r') as f:
    app_content = f.read()

# Remove the test_commanders route
app_content = app_content.replace('''@app.route('/test_commanders')
def test_commanders():
    """Simple test endpoint to check commanders"""
    try:
        # Check if we can find legendary creatures
        commanders = df_clean[df_clean['type_line'].str.contains('Legendary Creature', case=False, na=False)]
        count = len(commanders)
        
        # Get first 5 as examples
        examples = commanders.head(5)['name'].tolist()
        
        return jsonify({
            'status': 'success',
            'total_commanders': count,
            'examples': examples,
            'total_cards': len(df_clean)
        })
    except Exception as e:
        return jsonify({
            'status': 'error',
            'error': str(e)
        })

''', '')

# Remove the commanders route if it exists
app_content = app_content.replace('''@app.route('/commanders')
def get_commanders():
    """Get list of all available commanders"""
    try:
        # Find all legendary creatures (potential commanders)
        commanders = df_clean[
            df_clean['type_line'].str.contains('Legendary Creature', case=False, na=False)
        ].copy()
        
        # Sort by name for easier browsing
        commanders = commanders.sort_values('name')
        
        commander_list = []
        for _, row in commanders.iterrows():
            commander_list.append({
                'name': row['name'],
                'colors': list(get_color_identity(row['colors'])),
                'type': row['type_line']
            })
        
        return jsonify({
            'commanders': commander_list,
            'total': len(commander_list)
        })
        
    except Exception as e:
        return jsonify({'error': str(e)})

''', '')

# Save the cleaned app.py
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Restored original Flask app")

✅ Restored original Flask app


In [34]:
# Create the restored template (back to text input but keeping mana symbols)
restored_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3; /* Light gray background */
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* Mana symbol styles */
        .mana-symbol {
            display: inline-block;
            width: 18px;
            height: 18px;
            border-radius: 50%;
            margin: 0 2px;
            text-align: center;
            line-height: 18px;
            font-size: 11px;
            font-weight: bold;
            vertical-align: middle;
            border: 1px solid rgba(0,0,0,0.2);
            box-shadow: 0 1px 2px rgba(0,0,0,0.1);
        }
        
        .mana-w { background: linear-gradient(135deg, #fffbd5, #f8f4c0); color: #333; }
        .mana-u { background: linear-gradient(135deg, #0e68ab, #1565c0); color: white; }
        .mana-b { background: linear-gradient(135deg, #2e2e2e, #1a1a1a); color: white; }
        .mana-r { background: linear-gradient(135deg, #d32f2f, #c62828); color: white; }
        .mana-g { background: linear-gradient(135deg, #388e3c, #2e7d32); color: white; }
        .mana-c { background: linear-gradient(135deg, #9e9e9e, #757575); color: white; }
        .mana-generic { background: linear-gradient(135deg, #e0e0e0, #bdbdbd); color: #333; }
        
        .mana-cost-display {
            display: inline-flex;
            align-items: center;
            gap: 1px;
        }
        
        /* Fixed 3-column grid layout with consistent sizing */
        .results-grid { 
            display: grid; 
            grid-template-columns: repeat(3, 1fr);
            gap: 20px; 
            padding: 0;
            width: 100%;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080; /* Medium gray outline */
            border-radius: 10px; 
            padding: 20px;
            text-align: center;
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
            height: 650px; /* Fixed height for all boxes */
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }
        
        .result-box:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .similarity-score {
            position: absolute;
            top: 15px;
            right: 15px;
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 6px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 700;
            z-index: 10;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            margin-bottom: 8px;
            margin-top: 0;
            line-height: 1.3;
            padding-right: 60px; /* Space for similarity score */
        }
        
        .card-meta {
            font-size: 12px;
            color: #666;
            margin-bottom: 6px;
            font-weight: 500;
        }
        
        .card-type {
            font-size: 12px;
            color: #555;
            margin-bottom: 8px;
            font-weight: 500;
        }
        
        .card-oracle-text {
            font-size: 11px;
            color: #444;
            line-height: 1.3;
            margin-bottom: 10px;
            text-align: left;
            background: #f9f9f9;
            padding: 8px;
            border-radius: 6px;
            border-left: 3px solid #667eea;
            max-height: 80px;
            overflow-y: auto;
        }
        
        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center;
            flex-grow: 1;
            align-items: center;
        }
        
        .card-image-container img {
            width: 266px; /* 33% bigger than 200px */
            height: 372px; /* 33% bigger than 280px */
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 3px 6px rgba(0,0,0,0.2);
            background: white;
        }
        
        .card-footer {
            margin-top: auto;
            position: relative;
            height: 30px;
            width: 100%;
        }
        
        .scryfall-link {
            position: absolute;
            bottom: 0;
            left: 0;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 11px;
            font-weight: 600;
            padding: 4px 8px;
            border: 1px solid #667eea;
            border-radius: 12px;
            transition: all 0.3s ease;
            display: inline-block;
        }
        
        .scryfall-link a:hover {
            background: #667eea;
            color: white;
            transform: translateY(-1px);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 266px;
            height: 372px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
        }
        
        /* Responsive design */
        @media (max-width: 1200px) {
            .results-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .results-grid {
                grid-template-columns: 1fr;
                gap: 15px;
            }
            
            .result-box {
                height: auto;
                min-height: 500px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .card-image-container img {
                width: 200px;
                height: 280px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        function createManaSymbols(manaString) {
            if (!manaString || manaString === 'N/A' || manaString === '') {
                return '<span style="color: #999; font-size: 12px;">Free</span>';
            }
            
            const symbols = manaString.match(/\\{[^}]+\\}/g) || [];
            if (symbols.length === 0) return manaString;
            
            const manaHtml = symbols.map(symbol => {
                const clean = symbol.replace(/[{}]/g, '');
                
                if (clean === 'W') return '<span class="mana-symbol mana-w">W</span>';
                if (clean === 'U') return '<span class="mana-symbol mana-u">U</span>';
                if (clean === 'B') return '<span class="mana-symbol mana-b">B</span>';
                if (clean === 'R') return '<span class="mana-symbol mana-r">R</span>';
                if (clean === 'G') return '<span class="mana-symbol mana-g">G</span>';
                if (clean === 'C') return '<span class="mana-symbol mana-c">C</span>';
                if (clean === 'X') return '<span class="mana-symbol mana-x">X</span>';
                if (/^\\d+$/.test(clean)) return `<span class="mana-symbol mana-generic">${clean}</span>`;
                
                return `<span class="mana-symbol mana-generic">${clean}</span>`;
            }).join('');
            
            return `<span class="mana-cost-display">${manaHtml}</span>`;
        }

        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 30 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}">`;
            } else {
                html += `<div class="image-placeholder">Image not available</div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="results-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="result-box">
                        <div class="similarity-score">${(card.similarity * 100).toFixed(1)}%</div>
                        
                        <div class="card-name">${card.name}</div>
                        
                        <div class="card-meta">Colors: ${card.colors.join(', ') || 'Colorless'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity}</div>
                        
                        <div class="card-type">${card.type}</div>
                        
                        <div class="card-oracle-text">${card.text}</div>
                        
                        <div class="card-image-container">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}">`;
                } else {
                    html += `<div class="image-placeholder">Image not available</div>`;
                }
                
                html += `</div>
                        
                        <div class="card-footer">`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">Scryfall</a></div>`;
                }
                
                html += `</div>
                    </div>`;
            });
            
            html += `</div></div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        // Allow Enter key to trigger search
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the restored template
with open('templates/index.html', 'w') as f:
    f.write(restored_template)

print("✅ Fully restored to working state!")
print("\nYou now have:")
print("- Text input search box (working)")
print("- 30 results by default")
print("- Mana symbols instead of text")
print("- 3-column grid layout")
print("- All the visual improvements")
print("\nRestart your Flask app - everything should work as before!")

✅ Fully restored to working state!

You now have:
- Text input search box (working)
- 30 results by default
- Mana symbols instead of text
- 3-column grid layout
- All the visual improvements

Restart your Flask app - everything should work as before!


In [35]:
# Read the current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Update the result-box CSS to left-align text
old_text_align = 'text-align: center;'
new_text_align = 'text-align: left;'

content = content.replace(old_text_align, new_text_align)

# Update card-name CSS to not need right padding since we're left-aligning
old_card_name_css = '''        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            margin-bottom: 8px;
            margin-top: 0;
            line-height: 1.3;
            padding-right: 60px; /* Space for similarity score */
        }'''

new_card_name_css = '''        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            margin-bottom: 8px;
            margin-top: 0;
            line-height: 1.3;
            padding-right: 60px; /* Space for similarity score */
        }'''

content = content.replace(old_card_name_css, new_card_name_css)

# Update the card image container to still center the image
old_image_container = '''        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center;
            flex-grow: 1;
            align-items: center;
        }'''

new_image_container = '''        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center; /* Keep images centered */
            flex-grow: 1;
            align-items: center;
        }'''

content = content.replace(old_image_container, new_image_container)

# Update the displayResults function to include price in the second line
old_card_meta = '<div class="card-meta">Colors: ${card.colors.join(\\', \\') || \\'Colorless\\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity}</div>'

new_card_meta = '<div class="card-meta">Colors: ${card.colors.join(\\', \\') || \\'Colorless\\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: $${card.price || \\'N/A\\'}</div>'

content = content.replace(old_card_meta, new_card_meta)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated text alignment and added price display!")

SyntaxError: unexpected character after line continuation character (3624838500.py, line 54)

In [37]:
# Read the current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Update the result-box CSS to left-align text
content = content.replace(
    'text-align: center;',
    'text-align: left;'
)

# Update the card image container to still center the image
old_image_container = '''        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center;
            flex-grow: 1;
            align-items: center;
        }'''

new_image_container = '''        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center; /* Keep images centered */
            flex-grow: 1;
            align-items: center;
        }'''

content = content.replace(old_image_container, new_image_container)

# Update the displayResults function to include price in the second line
# Find the specific line with the card-meta
import re
pattern = r'<div class="card-meta">Colors: \$\{card\.colors\.join\(.*?\)\}</div>'
replacement = '<div class="card-meta">Colors: ${card.colors.join(", ") || "Colorless"} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: $${card.price || "N/A"}</div>'

# Use a more targeted replacement
old_line = 'Colors: ${card.colors.join(\', \') || \'Colorless\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity}'
new_line = 'Colors: ${card.colors.join(\', \') || \'Colorless\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: $${card.price || \'N/A\'}'

content = content.replace(old_line, new_line)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated text alignment and added price display!")

✅ Updated text alignment and added price display!


In [38]:
# Read the current Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

# Update the find_recommendations function to include price
old_card_result = '''            card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', '')
            }'''

new_card_result = '''            card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': get_card_price(card_row)
            }'''

app_content = app_content.replace(old_card_result, new_card_result)

# Add a function to extract price from the prices field
price_function = '''
def get_card_price(row):
    """Extract USD price from the prices field"""
    try:
        if 'prices' not in row or pd.isna(row['prices']):
            return None
            
        prices_str = row['prices']
        
        # Parse the prices string (it's a JSON-like string)
        if isinstance(prices_str, str):
            try:
                import json
                prices_dict = json.loads(prices_str.replace("'", '"'))
            except:
                try:
                    prices_dict = ast.literal_eval(prices_str)
                except:
                    return None
        elif isinstance(prices_str, dict):
            prices_dict = prices_str
        else:
            return None
        
        # Get USD price
        usd_price = prices_dict.get('usd')
        if usd_price and usd_price != 'None':
            try:
                return f"{float(usd_price):.2f}"
            except:
                return usd_price
        
        return None
        
    except Exception as e:
        return None

'''

# Insert the price function before the find_recommendations function
app_content = app_content.replace(
    'def find_recommendations(',
    price_function + 'def find_recommendations('
)

# Also update the commander info to include price
old_commander_result = '''        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', '')
        },'''

new_commander_result = '''        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },'''

app_content = app_content.replace(old_commander_result, new_commander_result)

# Save the updated Flask app
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Updated Flask app to include pricing!")

✅ Updated Flask app to include pricing!


In [41]:
# Read the template again to add price styling
with open('templates/index.html', 'r') as f:
    content = f.read()

# Add CSS for price styling
price_css = '''
        .price-display {
            color: #2e7d32;
            font-weight: 600;
        }
        
        .price-na {
            color: #999;
            font-style: italic;
        }'''

# Insert the CSS before the responsive design section
content = content.replace(
    '        /* Responsive design */',
    price_css + '\n\n        /* Responsive design */'
)

# Add the JavaScript function for price formatting
price_js = '''
        
        function formatPrice(price) {
            if (!price || price === 'N/A' || price === null) {
                return '<span class="price-na">N/A</span>';
            }
            return '<span class="price-display">$' + price + '</span>';
        }'''

# Insert before the createManaSymbols function
content = content.replace(
    '        function createManaSymbols(',
    price_js + '\n\n        function createManaSymbols('
)

# Update the card meta to use the formatting function (avoiding complex escaping)
# Find and replace the price part specifically
content = content.replace(
    'Price: $${card.price || \'N/A\'}',
    'Price: ${formatPrice(card.price)}'
)

# Save the final template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Added price formatting and styling!")
print("\nChanges made:")
print("- Left-aligned all text in result boxes")
print("- Similarity score remains in top right corner") 
print("- Added USD price to the second line with colors/mana/rarity")
print("- Images remain centered")
print("- Price displays in green when available, gray when N/A")
print("\nRestart your Flask app to see the changes!")

✅ Added price formatting and styling!

Changes made:
- Left-aligned all text in result boxes
- Similarity score remains in top right corner
- Added USD price to the second line with colors/mana/rarity
- Images remain centered
- Price displays in green when available, gray when N/A

Restart your Flask app to see the changes!


In [42]:
# First, let's check what price data we actually have
import pandas as pd

df_clean = pd.read_csv('data/mtg_cards_clean.csv', low_memory=False)

print("Checking price data availability...")
print(f"Total cards: {len(df_clean)}")

# Check if prices column exists
if 'prices' in df_clean.columns:
    print("✅ 'prices' column found")
    
    # Look at some sample price data
    sample_prices = df_clean['prices'].dropna().head(5)
    print(f"\nSample price data:")
    for i, price in enumerate(sample_prices):
        print(f"{i+1}. {str(price)[:100]}...")
        
    # Check how many cards have price data
    non_null_prices = df_clean['prices'].notna().sum()
    print(f"\nCards with price data: {non_null_prices}/{len(df_clean)}")
    
else:
    print("❌ 'prices' column not found")
    print("Available columns:", list(df_clean.columns))

# Check for other price-related columns
price_columns = [col for col in df_clean.columns if 'price' in col.lower() or 'usd' in col.lower()]
print(f"\nPrice-related columns found: {price_columns}")

# Sample a few cards to see their price data
if price_columns:
    for col in price_columns:
        sample_data = df_clean[col].dropna().head(3)
        print(f"\nSample {col} data:")
        for val in sample_data:
            print(f"  {val}")

Checking price data availability...
Total cards: 27290
✅ 'prices' column found

Sample price data:
1. {'usd': '0.04', 'usd_foil': '0.08', 'usd_etched': None, 'eur': '0.07', 'eur_foil': '0.06', 'tix': '0...
2. {'usd': '0.06', 'usd_foil': '0.13', 'usd_etched': None, 'eur': '0.15', 'eur_foil': '0.33', 'tix': '0...
3. {'usd': '0.19', 'usd_foil': '0.21', 'usd_etched': None, 'eur': '0.02', 'eur_foil': '0.02', 'tix': '0...
4. {'usd': None, 'usd_foil': '3.59', 'usd_etched': None, 'eur': None, 'eur_foil': '3.47', 'tix': '0.25'...
5. {'usd': '0.07', 'usd_foil': '0.10', 'usd_etched': None, 'eur': '0.07', 'eur_foil': '0.10', 'tix': '0...

Cards with price data: 27290/27290

Price-related columns found: ['prices']

Sample prices data:
  {'usd': '0.04', 'usd_foil': '0.08', 'usd_etched': None, 'eur': '0.07', 'eur_foil': '0.06', 'tix': '0.03'}
  {'usd': '0.06', 'usd_foil': '0.13', 'usd_etched': None, 'eur': '0.15', 'eur_foil': '0.33', 'tix': '0.32'}
  {'usd': '0.19', 'usd_foil': '0.21', 'usd_etched': 

In [43]:
# Update the Flask app with a better price extraction function
price_function_fixed = '''
def get_card_price(row):
    """Extract USD price from available price fields"""
    try:
        # Try different possible price fields
        price_fields = ['usd', 'prices']
        
        for field in price_fields:
            if field in row and pd.notna(row[field]):
                price_data = row[field]
                
                # If it's already a simple number/string
                if field == 'usd':
                    try:
                        price_val = float(price_data)
                        return f"{price_val:.2f}"
                    except:
                        return str(price_data) if price_data else None
                
                # If it's the prices field (JSON-like)
                elif field == 'prices':
                    if isinstance(price_data, str):
                        try:
                            # Try to parse as JSON
                            import json
                            prices_dict = json.loads(price_data.replace("'", '"'))
                            usd_price = prices_dict.get('usd')
                            if usd_price and usd_price != 'None' and usd_price != None:
                                return f"{float(usd_price):.2f}"
                        except:
                            try:
                                # Try literal_eval
                                prices_dict = ast.literal_eval(price_data)
                                usd_price = prices_dict.get('usd')
                                if usd_price and usd_price != 'None' and usd_price != None:
                                    return f"{float(usd_price):.2f}"
                            except:
                                pass
        
        return None
        
    except Exception as e:
        print(f"Price extraction error: {e}")
        return None

'''

# Read and update the Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

# Replace the old price function
old_function_start = 'def get_card_price(row):'
old_function_end = 'def find_recommendations('

# Find the old function and replace it
import re
pattern = r'def get_card_price\(row\):.*?(?=def find_recommendations)'
app_content = re.sub(pattern, price_function_fixed, app_content, flags=re.DOTALL)

# Save the updated app
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Updated price extraction function!")

✅ Updated price extraction function!


In [44]:
# Create a simple test to check if prices work
test_price_code = '''
@app.route('/test_prices')
def test_prices():
    """Test endpoint to check price extraction"""
    try:
        # Get a few cards and test price extraction
        sample_cards = df_clean.head(10)
        results = []
        
        for _, card in sample_cards.iterrows():
            price = get_card_price(card)
            results.append({
                'name': card['name'],
                'price': price,
                'raw_price_data': str(card.get('prices', 'No prices field'))[:100]
            })
        
        return jsonify({
            'status': 'success',
            'sample_prices': results
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'error': str(e)
        })

'''

# Add test to Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

app_content = app_content.replace(
    '@app.route(\'/\')',
    test_price_code + '\n@app.route(\'/\')'
)

with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Added price test endpoint!")
print("\nTo debug:")
print("1. Restart your Flask app")
print("2. Visit http://localhost:5001/test_prices")
print("3. This will show what price data is available")

✅ Added price test endpoint!

To debug:
1. Restart your Flask app
2. Visit http://localhost:5001/test_prices
3. This will show what price data is available


In [46]:
# Create the correct price extraction function
correct_price_function = '''
def get_card_price(row):
    """Extract USD price from the prices field"""
    try:
        if 'prices' not in row or pd.isna(row['prices']):
            return None
            
        prices_str = row['prices']
        
        # Parse the string representation of the dictionary
        if isinstance(prices_str, str):
            try:
                # Use ast.literal_eval to safely parse the dictionary string
                prices_dict = ast.literal_eval(prices_str)
                
                # Get USD price
                usd_price = prices_dict.get('usd')
                
                # Check if USD price exists and is not None
                if usd_price is not None and usd_price != 'None':
                    try:
                        # Convert to float and format to 2 decimal places
                        price_float = float(usd_price)
                        return f"{price_float:.2f}"
                    except (ValueError, TypeError):
                        return str(usd_price)
                
                # If no USD price, try USD foil as backup
                usd_foil_price = prices_dict.get('usd_foil')
                if usd_foil_price is not None and usd_foil_price != 'None':
                    try:
                        price_float = float(usd_foil_price)
                        return f"{price_float:.2f}"
                    except (ValueError, TypeError):
                        return str(usd_foil_price)
                        
                return None
                
            except (ValueError, SyntaxError) as e:
                print(f"Error parsing prices for {row.get('name', 'unknown')}: {e}")
                return None
        
        return None
        
    except Exception as e:
        print(f"Price extraction error for {row.get('name', 'unknown')}: {e}")
        return None

'''

# Read and update the Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

# Find and replace the old price function
import re
# Find the start and end of the old function
start_pattern = r'def get_card_price\(row\):'
end_pattern = r'(?=\ndef [a-zA-Z_]|$)'

# Replace the old function with the new one
old_function_match = re.search(start_pattern + r'.*?' + end_pattern, app_content, re.DOTALL)
if old_function_match:
    app_content = app_content.replace(old_function_match.group(0), correct_price_function)
else:
    # If we can't find the old function, just add the new one before find_recommendations
    app_content = app_content.replace(
        'def find_recommendations(',
        correct_price_function + '\ndef find_recommendations('
    )

# Save the updated app
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Fixed price extraction function!")
print("\nThe function now:")
print("- Correctly parses the dictionary string format")
print("- Extracts 'usd' price first")
print("- Falls back to 'usd_foil' if no regular USD price")
print("- Formats prices to 2 decimal places")
print("- Handles None values properly")
print("\nRestart your Flask app - prices should now display correctly!")

✅ Fixed price extraction function!

The function now:
- Correctly parses the dictionary string format
- Extracts 'usd' price first
- Falls back to 'usd_foil' if no regular USD price
- Formats prices to 2 decimal places
- Handles None values properly

Restart your Flask app - prices should now display correctly!


In [47]:
# Update the price function with debugging
debug_price_function = '''
def get_card_price(row):
    """Extract USD price from the prices field with debugging"""
    try:
        card_name = row.get('name', 'unknown')
        
        if 'prices' not in row:
            print(f"DEBUG: No 'prices' field for {card_name}")
            return None
            
        if pd.isna(row['prices']):
            print(f"DEBUG: 'prices' field is NaN for {card_name}")
            return None
            
        prices_str = row['prices']
        print(f"DEBUG: Raw prices for {card_name}: {str(prices_str)[:100]}")
        
        # Parse the string representation of the dictionary
        if isinstance(prices_str, str):
            try:
                # Use ast.literal_eval to safely parse the dictionary string
                prices_dict = ast.literal_eval(prices_str)
                print(f"DEBUG: Parsed prices dict for {card_name}: {prices_dict}")
                
                # Get USD price
                usd_price = prices_dict.get('usd')
                print(f"DEBUG: USD price for {card_name}: {usd_price} (type: {type(usd_price)})")
                
                # Check if USD price exists and is not None
                if usd_price is not None and usd_price != 'None' and usd_price != '':
                    try:
                        # Convert to float and format to 2 decimal places
                        price_float = float(usd_price)
                        result = f"{price_float:.2f}"
                        print(f"DEBUG: Final price for {card_name}: {result}")
                        return result
                    except (ValueError, TypeError) as e:
                        print(f"DEBUG: Error converting {usd_price} to float for {card_name}: {e}")
                        return str(usd_price)
                
                print(f"DEBUG: No valid USD price for {card_name}, usd_price was: {usd_price}")
                return None
                
            except (ValueError, SyntaxError) as e:
                print(f"DEBUG: Error parsing prices for {card_name}: {e}")
                return None
        else:
            print(f"DEBUG: Prices is not a string for {card_name}, type: {type(prices_str)}")
        
        return None
        
    except Exception as e:
        print(f"DEBUG: Price extraction error for {row.get('name', 'unknown')}: {e}")
        return None

'''

# Read and update the Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

# Replace the price function
# Find the existing function and replace it
start_marker = 'def get_card_price(row):'
end_marker = 'def find_recommendations('

start_pos = app_content.find(start_marker)
end_pos = app_content.find(end_marker)

if start_pos != -1 and end_pos != -1:
    # Replace the function
    new_content = app_content[:start_pos] + debug_price_function + app_content[end_pos:]
    
    with open('app.py', 'w') as f:
        f.write(new_content)
    
    print("✅ Added debugging to price function!")
    print("\nNow:")
    print("1. Restart your Flask app")
    print("2. Search for a commander")
    print("3. Check the terminal where Flask is running")
    print("4. You'll see debug output showing what's happening with prices")
    
else:
    print("❌ Could not find the price function to replace")
    print("Let me add the function in a different way...")
    
    # Add the function before find_recommendations
    app_content = app_content.replace(
        'def find_recommendations(',
        debug_price_function + '\ndef find_recommendations('
    )
    
    with open('app.py', 'w') as f:
        f.write(app_content)
    
    print("✅ Added debugging price function!")

✅ Added debugging to price function!

Now:
1. Restart your Flask app
2. Search for a commander
3. Check the terminal where Flask is running
4. You'll see debug output showing what's happening with prices


In [48]:
# The issue might be that the price isn't being passed correctly from Flask to JavaScript
# Let's check the JavaScript function that displays the price

# Read the template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Let's fix the formatPrice function and make sure it's working
new_format_price = '''
        function formatPrice(price) {
            console.log('formatPrice called with:', price, 'type:', typeof price);
            
            if (!price || price === 'N/A' || price === null || price === 'null' || price === undefined) {
                return '<span class="price-na">N/A</span>';
            }
            
            // Handle both string and number prices
            let priceStr = String(price);
            return '<span class="price-display">$' + priceStr + '</span>';
        }'''

# Replace the old formatPrice function
old_format_start = 'function formatPrice(price) {'
old_format_end = '        }'

# Find and replace the formatPrice function
import re
pattern = r'function formatPrice\(price\) \{[^}]*\}'
if re.search(pattern, content):
    content = re.sub(pattern, new_format_price.strip(), content)
else:
    # If we can't find it, add it before createManaSymbols
    content = content.replace(
        'function createManaSymbols(',
        new_format_price + '\n\n        function createManaSymbols('
    )

# Also let's add some debugging to the displayResults function
# Find the line where we build the card meta and add debugging
old_card_building = 'data.recommendations.forEach((card, index) => {'
new_card_building = '''data.recommendations.forEach((card, index) => {
                console.log('Card:', card.name, 'Price:', card.price);'''

content = content.replace(old_card_building, new_card_building)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Fixed frontend price display with debugging!")
print("\nNow:")
print("1. Restart your Flask app")
print("2. Search for a commander") 
print("3. Open browser Developer Tools (F12)")
print("4. Look at the Console tab to see price debugging info")
print("5. You should see prices displaying correctly now!")

✅ Fixed frontend price display with debugging!

Now:
1. Restart your Flask app
2. Search for a commander
3. Open browser Developer Tools (F12)
4. Look at the Console tab to see price debugging info
5. You should see prices displaying correctly now!


In [49]:
# Create a clean, working template with proper price display
working_template_with_prices = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTG Commander Deck Recommender</title>
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }
        
        body { 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1400px; 
            margin: 0 auto; 
            padding: 20px; 
            background-color: #d3d3d3;
            line-height: 1.4;
        }
        
        .header { 
            text-align: center; 
            margin-bottom: 30px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        
        .header h1 { 
            margin: 0; 
            font-size: 2.5em; 
            font-weight: 700;
        }
        
        .header p { 
            margin: 15px 0 0 0; 
            font-size: 1.1em; 
            opacity: 0.9; 
        }
        
        .search-box { 
            text-align: center;
            margin-bottom: 30px; 
            background: white;
            padding: 30px;
            border-radius: 15px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }
        
        .search-box input { 
            padding: 15px 20px; 
            width: 400px; 
            max-width: 90%;
            font-size: 16px; 
            border: 2px solid #ddd;
            border-radius: 30px;
            margin-right: 15px;
            outline: none;
            transition: border-color 0.3s;
        }
        
        .search-box input:focus {
            border-color: #667eea;
        }
        
        .search-box button { 
            padding: 15px 30px; 
            font-size: 16px; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white; 
            border: none; 
            border-radius: 30px;
            cursor: pointer; 
            transition: transform 0.2s, box-shadow 0.2s;
            font-weight: 600;
        }
        
        .search-box button:hover { 
            transform: translateY(-2px); 
            box-shadow: 0 4px 12px rgba(102, 126, 234, 0.4);
        }
        
        .commander-section { 
            background: white;
            padding: 25px; 
            border-radius: 15px; 
            margin-bottom: 30px; 
            box-shadow: 0 4px 12px rgba(0,0,0,0.1);
        }
        
        .commander-card {
            display: flex;
            gap: 25px;
            align-items: flex-start;
        }
        
        .commander-image {
            flex-shrink: 0;
            background: #f8f8f8;
            border-radius: 12px;
            padding: 10px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .commander-image img {
            width: 200px;
            height: 280px;
            object-fit: contain;
            border-radius: 8px;
            background: white;
        }
        
        .commander-info {
            flex-grow: 1;
        }
        
        .commander-info h2 {
            margin-bottom: 15px;
            color: #333;
            font-size: 2em;
            font-weight: 700;
        }
        
        .commander-info p {
            margin-bottom: 10px;
            font-size: 16px;
        }
        
        .recommendations-section {
            margin-top: 30px;
        }
        
        .recommendations-header {
            text-align: center;
            margin-bottom: 25px;
            font-size: 1.8em;
            color: #333;
            font-weight: 600;
        }
        
        /* Mana symbol styles */
        .mana-symbol {
            display: inline-block;
            width: 18px;
            height: 18px;
            border-radius: 50%;
            margin: 0 2px;
            text-align: center;
            line-height: 18px;
            font-size: 11px;
            font-weight: bold;
            vertical-align: middle;
            border: 1px solid rgba(0,0,0,0.2);
            box-shadow: 0 1px 2px rgba(0,0,0,0.1);
        }
        
        .mana-w { background: linear-gradient(135deg, #fffbd5, #f8f4c0); color: #333; }
        .mana-u { background: linear-gradient(135deg, #0e68ab, #1565c0); color: white; }
        .mana-b { background: linear-gradient(135deg, #2e2e2e, #1a1a1a); color: white; }
        .mana-r { background: linear-gradient(135deg, #d32f2f, #c62828); color: white; }
        .mana-g { background: linear-gradient(135deg, #388e3c, #2e7d32); color: white; }
        .mana-c { background: linear-gradient(135deg, #9e9e9e, #757575); color: white; }
        .mana-generic { background: linear-gradient(135deg, #e0e0e0, #bdbdbd); color: #333; }
        
        .mana-cost-display {
            display: inline-flex;
            align-items: center;
            gap: 1px;
        }
        
        .price-display {
            color: #2e7d32;
            font-weight: 600;
        }
        
        .price-na {
            color: #999;
            font-style: italic;
        }
        
        .results-grid { 
            display: grid; 
            grid-template-columns: repeat(3, 1fr);
            gap: 20px; 
            padding: 0;
            width: 100%;
        }
        
        .result-box { 
            background: white;
            border: 2px solid #808080;
            border-radius: 10px; 
            padding: 20px;
            text-align: left; /* Left-aligned text */
            position: relative;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            transition: box-shadow 0.3s ease;
            height: 650px;
            display: flex;
            flex-direction: column;
            overflow: hidden;
        }
        
        .result-box:hover {
            box-shadow: 0 4px 8px rgba(0,0,0,0.15);
        }
        
        .similarity-score {
            position: absolute;
            top: 15px;
            right: 15px;
            background: linear-gradient(135deg, #28a745, #20c997);
            color: white;
            padding: 6px 10px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: 700;
            z-index: 10;
        }
        
        .card-name {
            font-weight: 700;
            font-size: 16px;
            color: #333;
            margin-bottom: 8px;
            margin-top: 0;
            line-height: 1.3;
            padding-right: 60px;
        }
        
        .card-meta {
            font-size: 12px;
            color: #666;
            margin-bottom: 6px;
            font-weight: 500;
        }
        
        .card-type {
            font-size: 12px;
            color: #555;
            margin-bottom: 8px;
            font-weight: 500;
        }
        
        .card-oracle-text {
            font-size: 11px;
            color: #444;
            line-height: 1.3;
            margin-bottom: 10px;
            text-align: left;
            background: #f9f9f9;
            padding: 8px;
            border-radius: 6px;
            border-left: 3px solid #667eea;
            max-height: 80px;
            overflow-y: auto;
        }
        
        .card-image-container {
            margin: 10px 0;
            display: flex;
            justify-content: center;
            flex-grow: 1;
            align-items: center;
        }
        
        .card-image-container img {
            width: 266px;
            height: 372px;
            object-fit: contain;
            border-radius: 8px;
            box-shadow: 0 3px 6px rgba(0,0,0,0.2);
            background: white;
        }
        
        .card-footer {
            margin-top: auto;
            position: relative;
            height: 30px;
            width: 100%;
        }
        
        .scryfall-link {
            position: absolute;
            bottom: 0;
            left: 0;
        }
        
        .scryfall-link a {
            color: #667eea;
            text-decoration: none;
            font-size: 11px;
            font-weight: 600;
            padding: 4px 8px;
            border: 1px solid #667eea;
            border-radius: 12px;
            transition: all 0.3s ease;
            display: inline-block;
        }
        
        .scryfall-link a:hover {
            background: #667eea;
            color: white;
            transform: translateY(-1px);
        }
        
        .loading { 
            text-align: center; 
            font-style: italic; 
            color: #666; 
            padding: 60px;
            font-size: 18px;
        }
        
        .error { 
            color: #dc3545; 
            background: #f8d7da; 
            padding: 20px; 
            border-radius: 12px; 
            margin: 20px 0;
            text-align: center;
            font-weight: 500;
        }
        
        .image-placeholder {
            display: flex;
            align-items: center;
            justify-content: center;
            width: 266px;
            height: 372px;
            background: linear-gradient(135deg, #f0f0f0, #e0e0e0);
            color: #999;
            font-size: 14px;
            text-align: center;
            border-radius: 8px;
            font-weight: 500;
        }
        
        @media (max-width: 1200px) {
            .results-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }
        
        @media (max-width: 768px) {
            body {
                padding: 15px;
            }
            
            .commander-card {
                flex-direction: column;
                text-align: center;
            }
            
            .commander-image img {
                width: 160px;
                height: 224px;
            }
            
            .search-box input {
                width: 100%;
                max-width: 300px;
                margin-bottom: 15px;
                margin-right: 0;
            }
            
            .results-grid {
                grid-template-columns: 1fr;
                gap: 15px;
            }
            
            .result-box {
                height: auto;
                min-height: 500px;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .card-image-container img {
                width: 200px;
                height: 280px;
            }
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🧙‍♂️ MTG Commander Deck Recommender</h1>
        <p>Enter a commander name to get AI-powered card recommendations with perfect color identity matching!</p>
    </div>

    <div class="search-box">
        <input type="text" id="commanderInput" placeholder="Enter commander name (e.g., Arcades, Isshin, Krenko)" />
        <button onclick="getRecommendations()">Get Recommendations</button>
    </div>

    <div id="results"></div>

    <script>
        function createManaSymbols(manaString) {
            if (!manaString || manaString === 'N/A' || manaString === '') {
                return '<span style="color: #999; font-size: 12px;">Free</span>';
            }
            
            const symbols = manaString.match(/\\{[^}]+\\}/g) || [];
            if (symbols.length === 0) return manaString;
            
            const manaHtml = symbols.map(symbol => {
                const clean = symbol.replace(/[{}]/g, '');
                
                if (clean === 'W') return '<span class="mana-symbol mana-w">W</span>';
                if (clean === 'U') return '<span class="mana-symbol mana-u">U</span>';
                if (clean === 'B') return '<span class="mana-symbol mana-b">B</span>';
                if (clean === 'R') return '<span class="mana-symbol mana-r">R</span>';
                if (clean === 'G') return '<span class="mana-symbol mana-g">G</span>';
                if (clean === 'C') return '<span class="mana-symbol mana-c">C</span>';
                if (clean === 'X') return '<span class="mana-symbol mana-x">X</span>';
                if (/^\\d+$/.test(clean)) return `<span class="mana-symbol mana-generic">${clean}</span>`;
                
                return `<span class="mana-symbol mana-generic">${clean}</span>`;
            }).join('');
            
            return `<span class="mana-cost-display">${manaHtml}</span>`;
        }
        
        function formatPrice(price) {
            if (!price || price === 'N/A' || price === null || price === 'null') {
                return '<span class="price-na">N/A</span>';
            }
            return `<span class="price-display">$${price}</span>`;
        }

        async function getRecommendations() {
            const commander = document.getElementById('commanderInput').value;
            const resultsDiv = document.getElementById('results');
            
            if (!commander.trim()) {
                resultsDiv.innerHTML = '<div class="error">Please enter a commander name</div>';
                return;
            }
            
            resultsDiv.innerHTML = '<div class="loading">🔍 Searching for recommendations...</div>';
            
            try {
                const response = await fetch('/recommend', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ commander: commander, num_recommendations: 30 })
                });
                
                const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }
                
                displayResults(data);
            } catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }
        }
        
        function displayResults(data) {
            const resultsDiv = document.getElementById('results');
            
            let html = `
                <div class="commander-section">
                    <div class="commander-card">
                        <div class="commander-image">`;
            
            if (data.commander.image_url) {
                html += `<img src="${data.commander.image_url}" alt="${data.commander.name}">`;
            } else {
                html += `<div class="image-placeholder">Image not available</div>`;
            }
            
            html += `
                        </div>
                        <div class="commander-info">
                            <h2>${data.commander.name}</h2>
                            <p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>`;
            
            if (data.commander.scryfall_url) {
                html += `<p><a href="${data.commander.scryfall_url}" target="_blank">View on Scryfall →</a></p>`;
            }
            
            html += `
                        </div>
                    </div>
                </div>
                
                <div class="recommendations-section">
                    <h3 class="recommendations-header">🎯 Recommended Cards (${data.recommendations.length} found)</h3>
                    <div class="results-grid">`;
            
            data.recommendations.forEach((card, index) => {
                html += `
                    <div class="result-box">
                        <div class="similarity-score">${(card.similarity * 100).toFixed(1)}%</div>
                        
                        <div class="card-name">${card.name}</div>
                        
                        <div class="card-meta">Colors: ${card.colors.join(', ') || 'Colorless'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: ${formatPrice(card.price)}</div>
                        
                        <div class="card-type">${card.type}</div>
                        
                        <div class="card-oracle-text">${card.text}</div>
                        
                        <div class="card-image-container">`;
                
                if (card.image_url) {
                    html += `<img src="${card.image_url}" alt="${card.name}">`;
                } else {
                    html += `<div class="image-placeholder">Image not available</div>`;
                }
                
                html += `</div>
                        
                        <div class="card-footer">`;
                
                if (card.scryfall_url) {
                    html += `<div class="scryfall-link"><a href="${card.scryfall_url}" target="_blank">Scryfall</a></div>`;
                }
                
                html += `</div>
                    </div>`;
            });
            
            html += `</div></div>`;
            
            resultsDiv.innerHTML = html;
        }
        
        document.getElementById('commanderInput').addEventListener('keypress', function(e) {
            if (e.key === 'Enter') {
                getRecommendations();
            }
        });
    </script>
</body>
</html>
'''

# Save the working template
with open('templates/index.html', 'w') as f:
    f.write(working_template_with_prices)

print("✅ Restored working template with price display!")
print("\nRestart your Flask app - everything should work now with prices showing correctly!")

✅ Restored working template with price display!

Restart your Flask app - everything should work now with prices showing correctly!


In [50]:
# Let's add debugging to see what's actually being sent to the frontend
# Update the Flask app to log what it's sending

with open('app.py', 'r') as f:
    content = f.read()

# Add debugging to the find_recommendations function
old_return = '''    return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images v1.1"
        }
    }'''

new_return = '''    # Debug: Check first few results
    for i, result in enumerate(results[:3]):
        print(f"DEBUG RESULT {i}: {result['name']} - Price: {result.get('price', 'NOT FOUND')}")
    
    final_response = {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images v1.1"
        }
    }
    
    print(f"DEBUG: Sending response with {len(results)} recommendations")
    return final_response'''

content = content.replace(old_return, new_return)

# Save the updated app
with open('app.py', 'w') as f:
    f.write(content)

print("✅ Added JSON response debugging!")

✅ Added JSON response debugging!


In [51]:
# Add debugging to the frontend to see what data is received
with open('templates/index.html', 'r') as f:
    template_content = f.read()

# Add debugging to the displayResults function
old_foreach = '''data.recommendations.forEach((card, index) => {
                html += `'''

new_foreach = '''data.recommendations.forEach((card, index) => {
                console.log(`Card ${index}: ${card.name}, Price: ${card.price}`);
                html += `'''

template_content = template_content.replace(old_foreach, new_foreach)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(template_content)

print("✅ Added frontend debugging!")

✅ Added frontend debugging!


In [52]:
# Let's also add a simple test endpoint to see the raw JSON
test_endpoint = '''
@app.route('/test_price_response')
def test_price_response():
    """Test what gets sent to frontend"""
    try:
        # Get one card and test the full process
        test_card = df_clean.iloc[0]
        price = get_card_price(test_card)
        
        return jsonify({
            'test_card': test_card['name'],
            'extracted_price': price,
            'raw_prices': str(test_card.get('prices', 'No prices'))[:200]
        })
    except Exception as e:
        return jsonify({'error': str(e)})

'''

# Add to Flask app
with open('app.py', 'r') as f:
    content = f.read()

content = content.replace(
    '@app.route(\'/\')',
    test_endpoint + '\n@app.route(\'/\')'
)

with open('app.py', 'w') as f:
    f.write(content)

print("✅ Added price test endpoint!")
print("\nNow:")
print("1. Restart Flask app")
print("2. Visit http://localhost:5001/test_price_response")
print("3. Search for a commander and watch the terminal")
print("4. Open browser console (F12) to see frontend logs")
print("5. This will show us exactly where the price data is getting lost!")

✅ Added price test endpoint!

Now:
1. Restart Flask app
2. Visit http://localhost:5001/test_price_response
3. Search for a commander and watch the terminal
4. Open browser console (F12) to see frontend logs
5. This will show us exactly where the price data is getting lost!


In [53]:
# The issue is in the Flask app - the price extraction is working but not being added to the JSON response
# Let's check and fix the card_result building

with open('app.py', 'r') as f:
    content = f.read()

# Look for where we build the card_result dictionary and make sure price is included
# The issue might be that we're not actually adding the price to the result

# Let's find the card_result building section and make sure it includes price
old_card_result_pattern = '''card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': get_card_price(card_row)
            }'''

# Let's add explicit debugging to the card building
new_card_result_pattern = '''extracted_price = get_card_price(card_row)
            print(f"DEBUG: Building result for {card_row['name']}, extracted price: {extracted_price}")
            
            card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': extracted_price
            }
            
            print(f"DEBUG: Final card_result price for {card_row['name']}: {card_result.get('price', 'NOT IN DICT')}")'''

content = content.replace(old_card_result_pattern, new_card_result_pattern)

# If that pattern wasn't found, let's try a different approach
if old_card_result_pattern not in content:
    print("Pattern not found, let's search for the card building section differently...")
    
    # Look for the results.append line and add debugging before it
    if 'results.append(card_result)' in content:
        content = content.replace(
            'results.append(card_result)',
            '''print(f"DEBUG: About to append card_result: {card_result.get('name', 'unknown')} with price: {card_result.get('price', 'NO PRICE FIELD')}")
            results.append(card_result)'''
        )

# Save the updated app
with open('app.py', 'w') as f:
    f.write(content)

print("✅ Added detailed debugging to card result building!")
print("\nRestart Flask and search for a commander.")
print("Check the terminal output to see exactly what's happening with the price field.")

Pattern not found, let's search for the card building section differently...
✅ Added detailed debugging to card result building!

Restart Flask and search for a commander.
Check the terminal output to see exactly what's happening with the price field.


In [54]:
# Let's completely rewrite the section where we build card results
with open('app.py', 'r') as f:
    content = f.read()

# Find the section where we build recommendations and replace it entirely
# Look for the pattern where we loop through similar cards

old_loop_section = '''        # Only include cards with some similarity and legal color identity
        if similarity > 0 and is_legal_in_deck(card_row['colors'], commander_colors):
            card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': get_card_price(card_row)
            }
            results.append(card_result)'''

new_loop_section = '''        # Only include cards with some similarity and legal color identity
        if similarity > 0 and is_legal_in_deck(card_row['colors'], commander_colors):
            # Extract price for this card
            card_price = get_card_price(card_row)
            print(f"DEBUG: Extracted price for {card_row['name']}: {card_price}")
            
            card_result = {
                'name': card_row['name'],
                'similarity': float(similarities[idx]),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': card_price
            }
            
            print(f"DEBUG: Built card_result for {card_row['name']} with price: {card_result['price']}")
            results.append(card_result)'''

# Replace the section
content = content.replace(old_loop_section, new_loop_section)

# If that didn't work, let's try a more targeted approach
if old_loop_section not in content:
    print("Exact pattern not found, trying line-by-line replacement...")
    
    # Let's replace just the card_result building part
    lines = content.split('\n')
    new_lines = []
    in_card_result = False
    
    for line in lines:
        if 'card_result = {' in line:
            in_card_result = True
            new_lines.append('            # Extract price for this card')
            new_lines.append('            card_price = get_card_price(card_row)')
            new_lines.append(f'            print(f"DEBUG: Extracted price for {{card_row[\'name\']}}: {{card_price}}")')
            new_lines.append('')
            new_lines.append(line)
        elif in_card_result and "'price'" in line:
            # Replace the price line
            new_lines.append("                'price': card_price")
            in_card_result = False
        elif in_card_result and line.strip() == '}':
            new_lines.append("                'price': card_price")
            new_lines.append(line)
            new_lines.append('')
            new_lines.append(f'            print(f"DEBUG: Built card_result for {{card_row[\'name\']}} with price: {{card_result[\'price\']}}")')
            in_card_result = False
        else:
            new_lines.append(line)
    
    content = '\n'.join(new_lines)

# Save the updated content
with open('app.py', 'w') as f:
    f.write(content)

print("✅ Fixed card result building to properly include price!")
print("\nRestart Flask and test again - prices should now be included in the JSON response!")

Exact pattern not found, trying line-by-line replacement...
✅ Fixed card result building to properly include price!

Restart Flask and test again - prices should now be included in the JSON response!


In [55]:
# Let's create a clean working Flask app with proper price support
clean_flask_app = '''
from flask import Flask, render_template, request, jsonify
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast
import json

app = Flask(__name__)

# Color identity functions
def get_color_identity(colors_string):
    if pd.isna(colors_string) or colors_string == '[]':
        return set()
    try:
        colors_list = ast.literal_eval(colors_string)
        return set(colors_list) if colors_list else set()
    except:
        return set()

def is_legal_in_deck(card_colors_string, commander_colors_string):
    card_identity = get_color_identity(card_colors_string)
    commander_identity = get_color_identity(commander_colors_string)
    return card_identity.issubset(commander_identity)

def get_card_image_url(row, size='normal'):
    """Extract card image URL from the image_uris field"""
    try:
        if 'image_uris' not in row or pd.isna(row['image_uris']):
            return None
            
        image_uris = row['image_uris']
        
        if isinstance(image_uris, str):
            try:
                image_dict = ast.literal_eval(image_uris)
            except:
                try:
                    image_dict = json.loads(image_uris)
                except:
                    return None
        elif isinstance(image_uris, dict):
            image_dict = image_uris
        else:
            return None
        
        size_priority = [size, 'normal', 'small', 'large']
        for sz in size_priority:
            if sz in image_dict:
                return image_dict[sz]
        
        return list(image_dict.values())[0] if image_dict else None
            
    except Exception as e:
        print(f"Error getting image for {row.get('name', 'unknown')}: {e}")
        return None

def get_card_price(row):
    """Extract USD price from the prices field"""
    try:
        if 'prices' not in row or pd.isna(row['prices']):
            return None
            
        prices_str = row['prices']
        
        if isinstance(prices_str, str):
            try:
                prices_dict = ast.literal_eval(prices_str)
                
                usd_price = prices_dict.get('usd')
                
                if usd_price is not None and usd_price != 'None' and usd_price != '':
                    try:
                        price_float = float(usd_price)
                        return f"{price_float:.2f}"
                    except (ValueError, TypeError):
                        return str(usd_price)
                
                return None
                
            except (ValueError, SyntaxError) as e:
                return None
        
        return None
        
    except Exception as e:
        return None

# Load enhanced model
print("Loading enhanced ML model...")
try:
    with open('data/mtg_model_enhanced.pkl', 'rb') as f:
        model_data = pickle.load(f)
    
    df_clean = model_data['df_clean']
    keyword_matrix = model_data['keyword_matrix']
    keywords = model_data['keywords']
    
    print(f"Enhanced model loaded!")
    print(f"  - {len(df_clean)} cards")
    print(f"  - {len(keywords)} keywords")
    
except FileNotFoundError:
    print("Enhanced model not found, using basic model...")
    with open('data/mtg_model.pkl', 'rb') as f:
        model_data = pickle.load(f)
    df_clean = model_data['df_clean']
    keyword_matrix = model_data['keyword_matrix']
    keywords = model_data['keywords']

def find_recommendations(commander_name, num_recommendations=10):
    """Enhanced recommendation function with prices"""
    card_matches = df_clean[df_clean['name'].str.contains(commander_name, case=False, na=False)]
    
    if len(card_matches) == 0:
        return {"error": f"Commander '{commander_name}' not found"}
    
    commander_row = card_matches.iloc[0]
    commander_idx = card_matches.index[0]
    commander_colors = commander_row['colors']
    commander_vector = keyword_matrix[commander_idx].reshape(1, -1)
    
    similarities = cosine_similarity(commander_vector, keyword_matrix).flatten()
    all_indices = similarities.argsort()[::-1]
    
    results = []
    for idx in all_indices:
        if idx == commander_idx:
            continue
            
        card_row = df_clean.iloc[idx]
        similarity = similarities[idx]
        
        if similarity > 0 and is_legal_in_deck(card_row['colors'], commander_colors):
            # Get price for this card
            card_price = get_card_price(card_row)
            
            card_result = {
                'name': card_row['name'],
                'similarity': float(similarity),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': card_price
            }
            
            results.append(card_result)
            
            if len(results) >= num_recommendations:
                break
    
    return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images and Prices"
        }
    }

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    commander = data.get('commander', '')
    num_recs = data.get('num_recommendations', 10)
    
    result = find_recommendations(commander, num_recs)
    return jsonify(result)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5001)
'''

# Save the clean Flask app
with open('app.py', 'w') as f:
    f.write(clean_flask_app)

print("✅ Created clean Flask app with proper price support!")
print("\nThis version:")
print("- Has proper syntax with no errors")
print("- Includes price extraction in the card results")
print("- Has clean, readable code structure")
print("- Should display prices correctly")
print("\nRestart your Flask app now - it should work without syntax errors!")

✅ Created clean Flask app with proper price support!

This version:
- Has proper syntax with no errors
- Includes price extraction in the card results
- Has clean, readable code structure
- Should display prices correctly

Restart your Flask app now - it should work without syntax errors!


In [56]:
# Read the current template
with open('templates/index.html', 'r') as f:
    content = f.read()

# Change the default number of recommendations from 30 to 300
content = content.replace(
    'num_recommendations: 30',
    'num_recommendations: 300'
)

# Update the card meta line to remove "Colors: " since mana symbols show the info
old_meta_line = 'Colors: ${card.colors.join(\', \') || \'Colorless\'} | Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: ${formatPrice(card.price)}'

new_meta_line = 'Mana: ${createManaSymbols(card.mana_cost)} | ${card.rarity} | Price: ${formatPrice(card.price)}'

content = content.replace(old_meta_line, new_meta_line)

# Add price to the commander info section
old_commander_info = '''<p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>'''

new_commander_info = '''<p><strong>Type:</strong> ${data.commander.type}</p>
                            <p><strong>Color Identity:</strong> ${data.commander.colors.join(', ') || 'Colorless'}</p>
                            <p><strong>Price:</strong> ${formatPrice(data.commander.price)}</p>
                            <p><strong>Oracle Text:</strong> ${data.commander.text}</p>'''

content = content.replace(old_commander_info, new_commander_info)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated template with changes!")

✅ Updated template with changes!


In [57]:
# Read the current Flask app
with open('app.py', 'r') as f:
    app_content = f.read()

# Update the default num_recommendations parameter
old_default = 'def find_recommendations(commander_name, num_recommendations=10):'
new_default = 'def find_recommendations(commander_name, num_recommendations=300):'

app_content = app_content.replace(old_default, new_default)

# Also update the route handler default
old_route_default = "num_recs = data.get('num_recommendations', 10)"
new_route_default = "num_recs = data.get('num_recommendations', 300)"

app_content = app_content.replace(old_route_default, new_route_default)

# Save the updated Flask app
with open('app.py', 'w') as f:
    f.write(app_content)

print("✅ Updated Flask app for 300 results!")

✅ Updated Flask app for 300 results!


In [58]:
# Read the template again to optimize for more results
with open('templates/index.html', 'r') as f:
    content = f.read()

# Add some performance optimizations for handling 300 cards
# Reduce the card height slightly to fit more on screen
old_height = 'height: 650px;'
new_height = 'height: 600px;'

content = content.replace(old_height, new_height)

# Adjust the card image size slightly for better grid performance
old_image_size = '''width: 266px;
            height: 372px;'''

new_image_size = '''width: 240px;
            height: 336px;'''

content = content.replace(old_image_size, new_image_size)

# Also update the image placeholder size
old_placeholder_size = '''width: 266px;
            height: 372px;'''

new_placeholder_size = '''width: 240px;
            height: 336px;'''

content = content.replace(old_placeholder_size, new_placeholder_size)

# Save the optimized template
with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Optimized layout for 300 results!")
print("\nChanges made:")
print("- Now displays up to 300 results (100 rows of 3 cards)")
print("- Removed redundant 'Colors:' text from card info")
print("- Added price display to commander box")
print("- Slightly optimized card sizes for better performance")
print("- Cards now show: 'Mana: [symbols] | rarity | Price: $X.XX'")
print("\nRestart your Flask app to see the changes!")

✅ Optimized layout for 300 results!

Changes made:
- Now displays up to 300 results (100 rows of 3 cards)
- Removed redundant 'Colors:' text from card info
- Added price display to commander box
- Slightly optimized card sizes for better performance
- Cards now show: 'Mana: [symbols] | rarity | Price: $X.XX'

Restart your Flask app to see the changes!


In [59]:
# Update the Flask app to show what error is happening
with open('app.py', 'r') as f:
    content = f.read()

# Update the recommend route to show the actual error
old_route = '''@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    commander = data.get('commander', '')
    num_recs = data.get('num_recommendations', 300)
    
    result = find_recommendations(commander, num_recs)
    return jsonify(result)'''

new_route = '''@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        data = request.get_json()
        commander = data.get('commander', '')
        num_recs = data.get('num_recommendations', 300)
        
        print(f"DEBUG: Received request for {commander} with {num_recs} recommendations")
        
        result = find_recommendations(commander, num_recs)
        
        print(f"DEBUG: Successfully generated {len(result.get('recommendations', []))} recommendations")
        
        return jsonify(result)
        
    except Exception as e:
        print(f"ERROR in recommend route: {str(e)}")
        import traceback
        traceback.print_exc()
        return jsonify({"error": f"Server error: {str(e)}"})'''

content = content.replace(old_route, new_route)

# Save the updated app
with open('app.py', 'w') as f:
    f.write(content)

print("✅ Added error debugging to Flask app!")

✅ Added error debugging to Flask app!


In [60]:
# Update the frontend to show more detailed error info
with open('templates/index.html', 'r') as f:
    template_content = f.read()

# Update the getRecommendations function to show better error messages
old_catch = '''} catch (error) {
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations. Please try again.</div>';
            }'''

new_catch = '''} catch (error) {
                console.error('Frontend error:', error);
                resultsDiv.innerHTML = '<div class="error">Error getting recommendations: ' + error.message + '</div>';
            }'''

template_content = template_content.replace(old_catch, new_catch)

# Also add debugging to see the response
old_response_handling = '''const data = await response.json();
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }'''

new_response_handling = '''if (!response.ok) {
                    throw new Error(`HTTP ${response.status}: ${response.statusText}`);
                }
                
                const data = await response.json();
                console.log('Received data:', data);
                
                if (data.error) {
                    resultsDiv.innerHTML = `<div class="error">${data.error}</div>`;
                    return;
                }'''

template_content = template_content.replace(old_response_handling, new_response_handling)

# Save the updated template
with open('templates/index.html', 'w') as f:
    f.write(template_content)

print("✅ Added frontend debugging!")

✅ Added frontend debugging!


In [61]:
# Let's also create a temporary version that starts with fewer results
with open('templates/index.html', 'r') as f:
    content = f.read()

# Temporarily change to 50 results to test if the issue is the large number
content = content.replace(
    'num_recommendations: 300',
    'num_recommendations: 50'
)

with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Temporarily set to 50 results for testing!")
print("\nNow:")
print("1. Restart your Flask app")
print("2. Try searching for a commander")
print("3. Check the terminal where Flask is running for error messages")
print("4. Check browser console (F12) for frontend errors")
print("5. Tell me what error messages you see!")

✅ Temporarily set to 50 results for testing!

Now:
1. Restart your Flask app
2. Try searching for a commander
3. Check the terminal where Flask is running for error messages
4. Check browser console (F12) for frontend errors
5. Tell me what error messages you see!


In [1]:
# Let's try increasing to a reasonable number that balances performance with comprehensiveness
with open('templates/index.html', 'r') as f:
    content = f.read()

# Increase to 100 results (which gives 33+ rows of cards)
content = content.replace(
    'num_recommendations: 300',
    'num_recommendations: 100'
)

with open('templates/index.html', 'w') as f:
    f.write(content)

print("✅ Updated to 100 results!")
print("\nTry this with your Flask app:")
print("- 100 results = about 33 rows of 3 cards each")
print("- Should give comprehensive recommendations without performance issues")
print("- If this works well, we can try 150 or 200 next")

✅ Updated to 100 results!

Try this with your Flask app:
- 100 results = about 33 rows of 3 cards each
- Should give comprehensive recommendations without performance issues
- If this works well, we can try 150 or 200 next


In [72]:
# Update the Flask app to handle NaN values properly
with open('app.py', 'r') as f:
    content = f.read()

# Add a function to clean NaN values before JSON serialization
nan_cleaner_function = '''
def clean_for_json(obj):
    """Clean NaN values from objects before JSON serialization"""
    import math
    
    if isinstance(obj, dict):
        return {k: clean_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_for_json(item) for item in obj]
    elif isinstance(obj, float) and (math.isnan(obj) or math.isinf(obj)):
        return None
    else:
        return obj

'''

# Insert the function before find_recommendations
content = content.replace(
    'def find_recommendations(',
    nan_cleaner_function + 'def find_recommendations('
)

# Update the card result building to handle NaN values
old_card_result = '''card_result = {
                'name': card_row['name'],
                'similarity': float(similarity),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', ''),
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': card_price
            }'''

new_card_result = '''card_result = {
                'name': card_row['name'],
                'similarity': float(similarity),
                'type': card_row['type_line'],
                'colors': list(get_color_identity(card_row['colors'])),
                'text': card_row['oracle_text'][:200] + "..." if len(card_row['oracle_text']) > 200 else card_row['oracle_text'],
                'mana_cost': card_row.get('mana_cost', '') if pd.notna(card_row.get('mana_cost')) else '',
                'rarity': card_row.get('rarity', 'unknown'),
                'image_url': get_card_image_url(card_row, 'normal'),
                'scryfall_url': card_row.get('scryfall_uri', ''),
                'price': card_price
            }
            
            # Clean any NaN values
            card_result = clean_for_json(card_result)'''

content = content.replace(old_card_result, new_card_result)

# Also update the final return to clean the entire response
old_return = '''return {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images and Prices"
        }
    }'''

new_return = '''response_data = {
        "commander": {
            "name": commander_row['name'],
            "colors": list(get_color_identity(commander_colors)),
            "type": commander_row['type_line'],
            "text": commander_row['oracle_text'],
            "image_url": get_card_image_url(commander_row, 'normal'),
            "scryfall_url": commander_row.get('scryfall_uri', ''),
            "price": get_card_price(commander_row)
        },
        "recommendations": results,
        "model_info": {
            "keywords_used": len(keywords),
            "version": "Enhanced Multi-Word Keywords with Images and Prices"
        }
    }
    
    # Clean the entire response of NaN values
    return clean_for_json(response_data)'''

content = content.replace(old_return, new_return)

# Save the updated Flask app
with open('app.py', 'w') as f:
    f.write(content)

print("✅ Fixed NaN value handling in Flask app!")
print("\nChanges made:")
print("- Added clean_for_json() function to handle NaN values")
print("- Applied cleaning to card results and final response")
print("- Properly handle NaN mana_cost values")
print("\nRestart your Flask app - the JSON error should be fixed!")

✅ Fixed NaN value handling in Flask app!

Changes made:
- Added clean_for_json() function to handle NaN values
- Applied cleaning to card results and final response
- Properly handle NaN mana_cost values

Restart your Flask app - the JSON error should be fixed!
